# Fine tuning

In [34]:
from openai import OpenAI
from getpass import getpass
from tqdm import tqdm
import re
from utils.get_text import get_hwp_text, get_pdf_text # pdf, hwp에서 텍스트를 추출하는 사용자 지정 함수 
from datetime import datetime
from collections import defaultdict 
import numpy as np
import time
import json
import tiktoken 
import os

In [35]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [55]:
# 파일 경로 설정
file_path = f"금융/[경남] 2025년 1인 자영업자 사회보험망(고용ㆍ산재보험료) 지원사업 공고.pdf"
# 지원사업명 저장 (지원경로를 분할하여 사용)
title = file_path.split('/')[1].split('.')[0]

# 파일 타입 저장 (지원경로를 분할하여 사용)
file_type = file_path.split('/')[1].split('.')[1]

# file_type이 pdf면 get_pdf_text, hwp면 get_hwp_text를 사용하여 텍스트 추출
if file_type == 'pdf':
    full_text = get_pdf_text(file_path)
elif file_type == 'hwp':
    full_text = get_hwp_text(file_path)

def preprocess_text(full_text):
    ## 제어 문자 제거
    full_text = re.sub(r'[\uf000-\uf999\x00-\x1F]', '', full_text)
    
    ## 여러 개의 공백을 하나로 줄임
    full_text = re.sub(r'\s+', ' ', full_text).strip()

    return full_text
    
cleaned_text = preprocess_text(full_text)
print(cleaned_text)

- 1 -1인 자영업자 사회보험망(고용·산재보험료) 지원사업 공고경상남도 내 1인 자영업자에 대한 고용보험료 및 산재보험료 지원을 통해 폐업과 산업재해 등으로부터 보호하고 사회안전망을 강화하기 위하여 2025년도 「소상공인 사회보험망 지원사업」을 공고하오니 많은 참여 바랍니다. 2025. 2. 12. 경남신용보증재단 이사장1 사업개요□ 신청자격 및 지원대상 : 고용보험 및 산재보험에 가입하고, 보험료를 납부한 실적이 있는 도내 1인 자영업자 ※ 사업자등록증상 공동사업자의 경우 대표 1인에 대해서만 지원 □ 지원제외 :  지원 기간 중 보험 자격상실* 및 미납 보험료** * 가입 후 6월 연속 보험료 미납시, 폐업(사업종료시), 근로자로 전환시 ** 다만, 미납 보험료를 지원 기간 내 일시 납부한 경우는 지원 가능 ‚ 경상남도 사회보험망 지원사업에 지원종료 이력이 있는 자 【참고】v ′자영업자 고용보험′이란? - 「고용산재보험료 징수법」 제49조의2(자영업자에 대한 특례)에 따라 사업주가 고용보험에 가입할 수 있는 제도 - (가입혜택) 비자발적 사유로 폐업 시 실업급여 지급, 직업능력개발훈련비 지원 v ′중소기업사업주 산재보험′이란? - 「산업재해보상보호법」 제124조(중소기업 사업주에 대한 특례)에 따라 사업주가 산재보험에 가입할 수 있는 제도 - (가입혜택) 업무상 사유로 부상, 질병, 사망 등의 재해를 입은 경우 산재보험급여 지급 □ 지원내용 ❍ 고용보험료 지원 : 납부한 월 고용보험료 전 등급(1~7등급) 20% 지원 【고용보험료 지원 예시(요율 2.25%)】 (기준: ′25.1월, 단위: 원)등급1등급2등급3등급4등급5등급6등급7등급기준보수액(월)1,820,0002,080,0002,340,0002,600,0002,860,0003,120,0003,380,000보험료(월)40,95046,80052,65058,50064,35070,20076,050지원율경남지원보험료의 20%정부지원보험료의 80%보험료의 60%보험료의 50% 본인부담액0010,53011,70

In [37]:
base_template = f'''
📌 {title} 요약 안내

🤖 사업 개요

✅ 사업자 유형

☑️ 지원업종

💡 공고특성

👥 지원대상

📅 신청기간

📝 신청방법

🚫 지원제외 대상

⭐️ 우대사항

👩 여성 대상

💁 청년 대상

⚠ 유의사항

📑 제출서류

📌 선정절차

💰 지원내용

📞 문의처

'''

In [38]:
# 정의한 비슷한 용어 사전
keyword_dictionary = {  
                        '사업자 유형' : ['개인사업자', '법인사업자', '창업(예비사업자 포함)', '재창업', '기존사업자'],
                        '지원업종' : ['자동차 및 부품 판매업','도매 및 상품 중개업','소매업(자동차 제외)','숙박업', '음식점업', '제조업','교육 서비스업', '협회 및 단체, 수리 및 기타 개인 서비스업','부동산업','전문, 과학 및 기술 서비스업','예술, 스포츠 및 여가관련 서비스업','정보통신업','농업, 임업 및 어업','건설업','운수 및 창고업','보건업 및 사회복지 서비스업','사업시설 관리, 사업 지원 및 임대 서비스업','금융 및 보험업','전기, 가스, 증기 및 공기 조절 공급업','광업','수도, 하수 및 폐기물 처리, 원료 재생업','가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동','공공행정, 국방 및 사회보장 행정','국제 및 외국기관'],
                        '정보통신업' : ['정보통신 서비스업', '정보통신기기 제조업', '소프트웨어 및 컴퓨터 관련 서비스업'],
                        '공고 특성' : ['청년 대상', '여성 대상', '소상공인', '대출', '마케팅·홍보', '보조금', '폐업', '고용지원', '시설·환경 개선', '입주·임대 지원', '희망리턴패키지', '고용유지지원금', '희망대출플러스', '두루누리지원금', '창업패키지', '노란우산공제', '이커머스 입점피해'],
                        '창업패키지' : ['예비창업패키지', '초기창업패키지', '청년창업사관학교'],
                        '신청기간' : ['신청 기간', '접수 기간', '자금 소진시까지'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상', '접수방법','접수'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '우대사항' : ['벤처기업', '여성기업', '사회적기업(인증)', '장애인기업', '이노비즈제도','메인비즈제도','가족친화인증기업', '수출유망중소기업','녹색기술인증기업','내일채움공제 가입기업','글로벌 강소기업', '기술혁신중소기업'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준', '지원 절차'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용', '지원조건'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의', '접수'],
                        '기타' : ['기타']
                        }

In [39]:
explanation = '''
- 중소기업은 중소기업기본법에 정의된 기준을 따르며 법인사업자, 기존사업자를 포함해주세요.
- 시설·환경 개선 자금은 기업 운영에 필요한 기계설비, 공장설립, 토지구입 등에 사용되는 자금, 원자재 구매, 생산, 판매 등 기업의 운영에 필요한 자금입니다.
- 청년은 만 19세 이상 39세 이하를 의미합니다. 
- 전용 대상은 특정 대상만 신청이 가능한 경우를 의미하고, 우대 조건은 특정 대상에게 가점을 부여하는 경우를 의미합니다.
- 공고에서 언급된 모든 금리 우대 항목을 추출해주세요. 
- 여성대상과 청년대상은 대상이 여성만 또는 청년만 되는 내용이 있는 경우에 해당하며 O,X로 구분해주세요.
- 만약 '변동없음'이라고 기재되어 있다면, 기존에 제공된 값을 찾아서 사용하세요. 즉, 당초(1,3,5,6,9월)이라면 기존 값을 그대로 추출하세요.
'''

In [40]:
template = f'''
당신은 정부 지원 사업 전문 분석가입니다. 

사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 마크다운 형식으로 작성해주세요

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}
참고 : {explanation}

각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
'''

In [41]:
# Fine tuning 데이터 로드
dataset = [
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '부천시공고제2025-191호2025년 소상공인 특례보증․이차보전 지원계획 공고 부천시 소재 소상공인에 대한 2025년도 특례보증․이차보전지원 계획을 다음과 같이 공고합니다. 2025년 1월 23일 부 천 시 장1. 사 업 명 : 2025년 소상공인 특례보증 지원 가. 출 연 금 : 12억 원 나. 보증규모 : 120억 원 ※상반기 60억원 / 하반기 60억원 분리운영 다. 보증대상 : 부천시 소재 소상공인 ▸사업자 등록일로부터 2개월 이상 경과 된 사업자 라. 보증한도 : 사업자별 최대 5,000만 원 ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수 ▸기존 보증금액 포함 5,000만원 이내 ▸기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황), 재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정 마. 특례보증 제외대상 - 지방세, 세외수입 등 체납이 있는 사업자 - 제한업종(금융업 ․ 유흥업 ․ 불법도박 및 사치 ․ 향락업종 등) 바. 특례보증 신청 및 접수 - 접수기간 ․ 상반기 : 2025. 1. 23. ~ 자금소진시까지 ․ 하반기 : 2025. 7. 1. ~ 자금소진시까지 - 접 수 처 : 경기신용보증재단 부천지점 ․ 연락처 : 1577-5900 ․ 주 소 : 부천시 원미구 소향로 217, 4층 - 준비서류 : 사업자등록증 사본, 매출증빙서류, 신분증 등 사. 특례보증 처리 절차 상담․접수➡ 심사(서류․현장)➡보증서발급➡ 자금대출경기신용보증재단경기신용보증재단경기신용보증재단은행2. 사 업 명 : 2025년 소상공인 이차보전 지원 가. 대상 : 소상공인 특례보증 대상자 나. 내용 : 특례보증과 연계하여 대출이자 연 2% 1년간 지원 ▸사업장 휴․폐업, 사업장 타 시․군 이전 시 지원 중지 및 환수 다. 협약은행 : 6개 관내 금융기관 ▸농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행 ○ 기타 자세한 문의는 경기신용보증재단 부천지점(☎ 1577-5900) 또는 부천시 지역경제과(☎ 032-625-2766)로 문의하시기 바랍니다.'},
        {'role' : 'assistant', 'content' : """
        📌 [경기] 부천시 2025년 소상공인 특례보증ㆍ이차보전 지원계획 공고 요약 안내
        
        🤖 사업 개요 
        - 부천시 소재 소상공인에 대한 2025년도 특례보증․이차보전지원 계획을 다음과 같이 공고합니다.
    
        ✅ 사업자 유형
        - 개인사업자
        - 기존사업자

        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고 특성
        - 소상공인
        - 대출

        👥 지원대상
        1. 2025년 소상공인 특례보증 지원 : 부천시 소재 소상공인
         - 사업자 등록일로부터 2개월 이상 경과 된 사업자
        2. 2025년 소상공인 이차보전 지원 : 소상공인 특례보증 대상자

        📅 신청기간
        - 상반기: 2025. 1. 23. ~ 자금 소진 시까지
        - 하반기: 2025. 7. 1. ~ 자금 소진 시까지

        📝 신청방법
        - 특례보증 접수처 : 경기신용보증재단 부천지점
        - 주소 : 부천시 원미구 소향로 217, 4층

        🚫 지원제외 대상
        - 지방세, 세외수입 등 체납이 있는 사업자
        - 제한업종(금융업, 유흥업, 불법도박 및 사치, 향락업종 등)

        ⭐️ 우대사항

        👩 여성 대상
        X

        💁 청년 대상 
        X

        ⚠ 유의사항

        📑 제출서류
        - 사업자등록증 사본
        - 매출증빙서류
        - 신분증 등

        📌 선정절차
        - 상담․접수 ➡ 심사(서류․현장) ➡ 보증서발급 ➡ 자금대출

        💰 지원내용
        1. 2025년 소상공인 특례보증 지원
        - 보증한도 : 사업자별 최대 5,000만 원
        * 사업장 휴·폐업, 사업장 타 시·군 이전 시 지원 중지 및 환수
        * 기존 보증금액 포함 5,000만원 이내
        * 기 대출상황, 신용도 및 가동상태(사업내용, 주요영업현황), 재무상황(시설수준 및 거래내역) 등 경기신용보증재단 부천지점의 종합적 심사를 통해 보증금액 결정
        2. 2025년 소상공인 이차보전 지원
        - 특례보증과 연계하여 대출이자 연 2% 1년간 지원
        * 사업장 휴·폐업, 사업장 타 시·군 이전 시 지원 중지 및 환수
        - 협약은행 : 6개 관내 금융기관
        * 농협은행, 하나은행, 국민은행, 기업은행, 신한은행, 우리은행

        📞 문의처
        - 경기신용보증재단 부천지점: 1577-5900
        - 부천시 지역경제과: 032-625-2766
    """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '경제일자리재단 공고 제2025-3호「’25년 1인 자영업자 및 소상공인 사회보험료 지원사업」 공고 경기침체로 어려움을 겪는 소상공인의 경영난 해소와 근로자 고용안정망 구축을 위하여 ‘1인 자영업자 및 소상공인 사회보험료 지원사업’의 참여자를 다음과 같이 모집 공고합니다. 2025년 2월 17일(재)광주경제진흥상생일자리재단 대표이사1 사업개요 ❍ 사 업 명 : 1인 자영업자 및 소상공인 사회보험료 지원사업 ❍ 사업기간 : 2025. 1. ~ 사업비 소진 시까지 ❍ 지원대상 - (1인자영업자) 고용 및 산재보험법에 따라 기(신규) 가입한 1인 자영업자 - (소상공인) 정부 두루누리 사업장으로 관내 10인 미만 근로자를 고용한 사업장의 사업주 ❍ 지원내용 - (1인자영업자) 부담한 고용보험료 부과액의 20%, 산재보험료 부과액의 50% - (소상공인) 근로자의 고용산재 보험료 중 정부지원을 제외한 사업주 부담분(고용보험 20%, 산재보험 100%) 지원 ❍ 수행기관 : (재)광주광역시 경제진흥상생일자리재단2 세부내용 1인 자영업자 사회보험료 지원사업 ❍ 사업기간 : ’25. 1. 1. ~ 사업비 소진 시까지 ❍ 지급절차 : 1인 자영업자가 월별 보험료 선납 -> 월별 납부내역 확인 후 지원금 지급 ❍ 지급시기 : 매월말까지 확정된 지원대상자에 대하여 월별 지급고용・산재보험 가입 및 납부➡온·오프라인 신청(기업지원시스템·이메일 접수, 방문접수)➡고용·산재보험 납부내역 확인 및 지급대상 검토➡사실관계 확인 후 보험료 지원금 지급1인자영업자1인자영업자→경제진흥상생일자리재단경 제 진 흥 상 생 일 자 리 재 단→근로복지공단경제진흥상생일자리재단→1인자영업자기납부연중연중월별 지급 * 예) 1월 사회보험료 지원금은 1월 보험료의 법정 납부기한(2월 10일경)의 익월인 3월 지급 ❍ 지급대상 : 고용 및 산재보험법에 따라 기(신규) 가입한 1인 자영업자 (고용보험 기준보수 1~7등급 / 산재보험 기준보수 1~12등급까지 전 등급 지원) * 광주소재사업장에 한하며, 근로복지공단의 보험 가입대상을 기본조건으로 함 공동사업자의 경우 대표 1인만 지원 ❍ 지급방법 : 사업주 명의 계좌로 지원금 지급 ❍ 신청방법 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청 * 온라인 신청 : https://gjbizinfo.or.kr(광주기업지원시스템) * 최초 1회 신청, 사업자 변경(사업주, 사업자등록번호 등)되는 경우 변경신청 필수 * 기신청했던 업체에서는 신청자 변동 없을 시 별도 신청 없이 지급 예정 * 오프라인 신청 : 방문접수(경제진흥상생일자리재단 3층 소상공인디지털전환실) ❍ 제출서류 : 자영업자 고용ž산재보험에 가입 후 아래의 서류 제출구 분 제출서류비고필수제출서류①사업신청서(행정정보공동이용동의서포함)[서식1]②중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]③통장사본(스마트폰캡처본도인정) 수수료없음선택제출서류①사업자등록증또는사업자등록증명원사본1부.②건강보험자격득실확인서사본1부.(신청일로부터1개월이내발급본)*국민건강보험공단홈페이지(www.nhis.go.kr)또는정부24에서발급가능행정정보공동이용미동의시별도제출 * 추가 서류제출이 필요한 경우v 사업자등록증의 상호와 건강보험자격득실확인서상 사업장명칭이 동일할 경우 :‘사 업 장 가 입 자 별 부 과 현 황’ 또 는 ‘개 인 별 건 강 보 험 고 지 산 출 내 역’ (최 근 2개 월 이 내 의 고 지 연 월 로 발 급) 제 출 소상공인 사회보험료 지원사업 ❍ 사업기간 : ’25. 1. 1. ~ 사업비 소진 시까지 ❍ 지급절차 : 사업자가 월별 보험료 선납 -> 월별 납부내역 확인 후 지원금 지급두루누리사회보험지원신청➡보험료先납부➡서류접수(온ž오프라인)➡고용·산재보험 납부내역 확인 및 지급대상 검토➡보험료 지원금지급사업주 ->근로복지공단사업주사업주 ->재단재단 <-> 근로복지공단재단 -> 사업주 ❍ 지급시기 : 매월말까지 확정된 지원대상자에 대하여 월별 지급 * 단, 1월 지원금은 1월 보험료의 법정 납부기한(2월 10일경)의 다음달인 3월부터 지급 * 두루누리 사회보험료 지원을 받은 해당연도에 한하여 市 사회보험료를 지원 하고 해당 월 보험료를 법정 납부기한까지 완납한 건만 지원 ❍ 지급대상 : 정부 두루누리 사업장으로 관내 10인 미만 근로자를 고용한 사업장의 사업주 * 지원제외 : 사업주 본인, 건설·벌목업, 예술인, 특수형태 근로종사자 사업장, 사업주의 배우자 및 직계존비속 근로자의 보험료 미지원 ❍ 지급조건 ① 근로자 수가 10명 미만인 사업장(사업주) * 두루누리 사회보험료 지원여부로 확인 ‧ 지원신청일이 속한 보험연도의 전년도에 근로자인 피보험자 수가 월평균 10명 미만이고 지원신청 월 말 기준 10명 미만인 사업장 ‧ 지원신청일이 속한 보험연도의 전년도에 근로자인 피보험자 수가 월평균 10명 이상이나 지원신청 월 직전 3개월 동안 연속 10명 미만인 사업장 ‧ 지원신청일이 속한 보험연도 중에 보험관계가 성립한 사업으로 지원신청 월 직전 3개월 동안 근로자인 피보험자 수가 연속 10명 미만인 사업장 ② 월평균 보수액 270만원 미만 근로자 고용 * 두루누리 사회보험료 지원여부로 확인 ‧ 「소득세법」에 따른 근로소득에서 비과세 근로소득을 공제한 총급여액을 기준으로 월 평균액이 270만원 미만 ‧ 신규가입자 근로복지공단 납부내역과 4대보험 사업장 가입자명부로 확인 ❍ 신청방법 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청 * 온라인 신청 : https://gjbizinfo.or.kr(광주기업지원시스템), 담당자 이메일 접수 * 기존에 신청했던 업체에서는 신청자 변동 없을 시 별도 신청없이 지급 예정 * 오프라인 신청 : 방문접수(경제진흥상생일자리재단 3층 소상공인디지털전환실) ❍ 제출서류구 분신청주기제출서류신규신청최초1회①소상공인고용·산재보험료지원신청서[서식2]②사회보험료지원체크리스트[서식3]③중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]④개인정보수집·활용동의서(근로자용)[첨부]/보험료지원신청근로자전체⑤사업자등록증,사업자등록증명원,고유번호증(비영리단체)사본⑥사업주또는사업장명의통장사본(스마트폰캡처본도인정)⑦4대보험사업장가입자명부변경신청근로자추가시(추가신청자만해당)①소상공인고용·산재보험료지원신청서[서식2]②사회보험료지원체크리스트[서식3]③개인정보수집·활용동의서(근로자용)[첨부]/변경신청근로자전체【참 고】○ 고용보험이란? - 「고용산재보험료 징수법」 제49조의2(자영업자에 대한 특례)에 따라, 사업주가 고용보험에 가입할 수 있는 제도 - (가입혜택) 비자발적 사유로 폐업 시 실업급여 지급, 직업능력개발훈련비 지원○ 산재보험이란? - 「산업재해보상보호법」 제124조(중소기업 사업주에 대한 특례)에 따라, 사업주가 산재보험에 가입할 수 있는 제도 - (가입혜택) 업무상 사유로 부상, 질병, 사망 등의 재해를 입은 경우 산재보험급여 지급3 문의처 ❍ 1인 자영업자 사회보험료 지원사업구분담당기관전화번호고용·산재보험 광주광역시 지원문의(재)경제진흥상생일자리재단 소상공인디지털전환실062-960-2632고용·산재보험 신규가입 문의근로복지공단1588-0075고용보험 정부지원 문의소상공인시장진흥공단1357 ❍ 소상공인 사회보험료 지원사업구분담당기관전화번호사회보험료광주광역시 지원문의(재)경제진흥상생일자리재단 소상공인디지털전환실062-960-2632두루누리 사회보험료지원문의근로복지공단1588-0075국민연금공단1355서식 1 25년 1인 자영업자 사회보험료(고용,산재보험) 지원신청서(앞쪽)접수번호접수일자신청인대표자명주민번호휴대전화업체명사업장주소사업자등록번호업종가입여부고용□ 가입 □ 미가입본인제외 상시근로자 유무□ 있음 □ 없음산재□ 가입 □ 미가입지원금지급계좌(사업주명의) 은행명예금주(신청인) 계좌번호 * 지원금 지급 계좌는 첨부 서류의 통장 사본과 반드시 일치하여야 함「소상공인 보호 및 지원에 관한 법률」 제12조의7 및 광주광역시 소상공인지원조례 제6조에 따라 위와 같이 신청합니다. 2025년 월 일신청인(서명 또는 인) (재)광주경제진흥상생일자리재단 대표이사 귀하필수제출서류1.사업신청서(행정정보공동이용동의서포함)2.통장사본(스마트폰캡처본도인정)3.중소기업통합관리시스템기업정보수집·이용·제공동의서수수료없음선택제출서류1.사업자등록증또는사업자등록증명원사본1부.2.건강보험자격득실확인서사본1부.(신청일로부터1개월이내발급본)*국민건강보험공단홈페이지(www.nhis.go.kr)또는정부24에서발급가능행정정보공동이용미동의시별도제출행정정보공동이용동의서 본인은 이 건 업무처리와 관련하여 「전자정부법」 제36조제1항에 따른 행정정보의 공동이용을 통하여 담당공무원이 위의 사항을 확인하는 것에 동의합니다. 신청인 (서명 또는 인)서식2 25년 사회보험료(고용, 산재보험) 지원 신청서 (□ 신규 □ 변경)※ 뒷면의 작성방법 및 유의사항을 반드시 확인하여 주시기 바랍니다. (앞면) 접수번호 접수일신청인업체명사업자등록번호사업장관리번호(*필수)(고용보험)(산재보험)사업장주소 대표자명/주민번호담당자명연락처(사무실)이메일주소 연락처(휴대폰)사회보험료 지원 신청 근로자연번성 명생년월일(8자리) 성별퇴사자 체크1234필수제출서류1.소상공인고용·산재보험료지원신청서[서식2]2.사회보험료지원부정수급방지체크리스트[서식3]3.중소기업통합관리시스템기업정보수집·이용·제공동의서[서식4]4.개인정보수집·활용동의서(근로자용)[첨부]/보험료지원신청근로자전체5.사업자등록증,사업자등록증명원,고유번호증(비영리단체)사본6.사업주또는사업장명의통장사본(스마트폰캡처본도인정)7.4대보험사업장가입자명부※서류는최초신청1회만제출,단,사업장변경(사업주,사업자등록번호,근로자등)되는경우변경신청필수선택제출서류사업자등록증행정정보공동이용미동의시별도제출(필수) 지원대상자 중 사업주(대표자), 사업주(대표자)의 배우자, 직계존비속이 없음을 확인하며, 신청내용에 허위나 거짓이 있을 경우나 부적격자에게 지원금이 지급되거나 잘못 지급된 경우, 지원대상자가 정부 및 지자체 등의 다른 보험료 지원과 중복 수령 시 지원금 전액을 환수하고, 민·형사상 책임을 지는 것에 동의합니다. ※ 동의하지 않는 경우 지원금이 지급되지 않습니다. (동 의 □ 부 동 의 □)(필수) 타지역으로 사업장을 이전한 경우, 지원금이 자동 종료됩니다. (확 인 □ ) 2025년 월 일신청인(사업주) (서명 또는 인)공동신청인(공동사업주) (서명 또는 인)(재)광주광역시 경제진흥상생일자리재단 대표이사 귀하행정정보 공동이용 동의서 본인은 이 건 업무처리와 관련하여 「전자정부법」 제36조제1항에 따른 행정정보의 공동이용을 통하여 담당공무원이 위의 사항을 확인하는 것에 동의합니다. 신청인 (서명 또는 인)작 성 방 법1.최초 지원 신청하는 사업장은 신규 란에 표기하여 주시고, 기존에 신청했던 사업장 중 변경사항이 있는 사업장은 변경란에 표기하여야 합니다.※ 기존 신청 사업장 중 변경이 없는 사업장은 표기하지 않습니다.2. 사회보험별 사업장 관리번호는 사업장에 따라 모두 동일한 경우도 있고, 각각 부여되어 있는 경우도 있으므로 신청인이 사회보험료 별로 확인 후 정확하게 기재해야 합니다. ※근로복지공단(1588-0075) 3. 지원 신청 대상자는 사업장 소속 근로자 중 월평균 보수액 270만원 미만으로 정부의 두루누리 사회보험 지원을 받고 있으며, 1개월 이상 고용을 유지하고 있는 근로자와 퇴사자(퇴사 전 1개월 이상 고용유지, 두루누리 사회보험 지원 필수)를 포함하여 기재해야 합니다.4. 공동사업주가 있을 경우 공동사업주 현황을 기재하여 주시고, 개인정보 수집‧이용 및 제3자 제공 동의서를 별도로 제출해야 합니다.유 의 사 항1.두루누리 사회보험 지원금을 받은 월에 한하여 광주광역시 사회보험료를 지원하며, 두루누리 사회보험 지원금은 해당 월 보험료를 법정 납부기한까지 완납하여야만 지원됩니다.※ 법정 납부기한 후에 보험료를 완납하여도 두루누리 사회보험 지원금은 지원되지 않습니다.2. 사회보험료가 체납된 사업장은 사회보험료 지원을 받을 수 없습니다.3. 신청 사업장의 신규 입사자가 사회보험료 지원 최초 신청 시 개인정보 수집‧이용 및 제3자 제공 동의서를 제출해야 하며, 퇴사자에 대한 지원 신청 시 퇴사자도 개인정보 수집‧이용 및 제3자 제공동의서를 제출해야 지원이 가능합니다.처 리 절 차신청서 등 작성및 제출è접수 및 확인è신청자별 보험료부과내역 확인è지원금 지급여부안내 및 지급è지원금 수령[신청인][경 제 진 흥 상 생일 자 리 재 단][광 주 광 역 시 및 근 로 복 지 공 단][경 제 진 흥 상 생일 자 리 재 단][신청인]서식3 사회보험료(고용, 산재보험) 지원 부정수급방지 체크리스트q 광주광역시 사회보험료 지원사업은 최저임금 인상에 따른 사업주의 경영부담 완화를 위해 사회보험료 사업주 부담분을 지원하는 사업입니다.q 본 지원금은 아래 사업장과 근로자 요건을 모두 충족하여야 지급됨을 알려 드리며, 요건 충족 시 굵은 선 안의 체크표시에 진하게 표시해 주시면 됩니다. (1개 문항이라도 “아니요”에 해당할 경우 지원 사업 참여 불가)q 사업장 요건 예아니오1. 사회보험료 지원 신청을 한 월부터 매월 말일 기준 근로자수가 10인 미만입니다.2. 10인 미만 요건 충족을 위해 인위적 감원이나 사업장을 분할하지 않았습니다.3. 임금체불 중 또는 임금체불로 명단 공개중이 아닙니다.4. 국가 및 공공기관에 해당하지 않습니다. 5. 두루누리 지원을 제외한 다른 사회보험료 지원을 받고 있지 않습니다. 6. 국가나 지자체로부터 최저임금 인상에 따라 추진되는 다른 지원 사업으로 재정지원을 받고 있지 않습니다. (일자리 안정자금 지원 제외)7. 근로자의 신규 입사 및 퇴사의 경우 각 사회보험별 자격 취득 및 상실 신고와 보수 변경 시 보수월액변경신고 등을 준수하고 있습니다. q 근로자 요건(반드시 개인별로 판단하여야 합니다.) 예아니오1.지원 신청대상 근로자는 월평균 보수액이 270만원 미만이며 정부의 두루누리 지원을 받고 있습니다.2. 지원 신청대상 근로자는 모두 1개월 이상 고용을 유지하고 있습니다.3. 지원 신청대상 근로자는 사업주, 사업주의 배우자 또는 사업주의 직계존비속이 아닙니다.4. 사회보험 가입대상인 근로자는 사회보험에 가입되어 있습니다. 5. 기존 근로자는 최소한 전년도 보수수준을 유지하고 있습니다.□ 유의사항 m 지원기간 동안 새로운 근로계약으로 인해 월평균 보수액이 270만원 이상이 된 근로자는 지원신청 하실 수 없습니다. 위 내용을 충분히 이해하였으며, 거짓이나 그 밖의 부정한 방법으로 광주광역시 사회보험료 지원 신청을 하여 지원금을 수령할 시 지원금 환수와 더불어 필요 시 민·형사상 책임을 질 수 있음에 동의합니다. 2025년 월 일신청인(사업주): (서명 또는 인) 공동신청인(공동사업주): (서명 또는 인)(재)광주광역시 경제진흥상생일자리재단 대표이사 귀하첨 부 사회보험료(고용, 산재보험) 지원신청을 위한개인정보 수집·이용 및 제3자 제공 동의서(근로자용)소상공인을 대상으로 고용・산재보험료의 일부를 지원하고 광주광역시 소상공인 지원사업 문자알림 서비스제공을 위하여 귀하의 개인정보를 수집·이용하고자 하며, 아울러 『고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률』에 따라 고용・산재보험에 가입한 근로복지공단으로부터 개인정보를 제공받고자 『개인정보 보호법』 제15조제1항제1호, 제17조제1항제1호, 제18조제2항 제1호, 제3항에 따라 귀하의 동의를 받고자 하오니, 아래의 내용을 자세히 읽어보신 후 동의 여부를 결정하여 주시기 바라며 동의를 거부할 경우 사회보험료 지원에 제한을 받을 수 있습니다.□ 개인정보 수집․이용 내역항 목 수 집 목 적보 유 기 간성명, 생년월일, 성별, 월 보수액 및 보수총액, 개인별 사회보험료 산출·부과·고지·납부 내역 및 사회보험료 정부지원내역, 소속 사업장 관리번호 등 광주광역시 사회보험료 신청(지원) 사업장의 지원 적합여부 및 지원 금액 확인해당 사업장의 광주광역시 사회보험료 지원 종료 시까지광주광역시 소상공인 지원사업 정보 제공2년□ 위와 같이 개인 및 사업장 정보를 수집·이용하는데 동의하십니까? 동의 미동의□ 개인 및 사업장 정보 제3자 제공내역제공하는 자제공받는 자제 공 항 목제 공 목 적보 유 기 간광주광역시,광주경제진흥상생일자리재단(수행기관)근로복지공단성명, 생년월일,사업장 관리번호신청 사업장이 부담하고 있는 근로자의 월 보수액, 사업장 부담 사회보험료 산출·부과·납부내역 및 사회보험료 지원 내역, 사회보험 취득·상실일 확인해당 사업장의 광주광역시 사회보험료 지원 종료 시까지성명, 연락처광주광역시 소상공인 지원사업 정보 문자알림2년□ 위와 같이 개인 정보를 수집·이용하고 제3자에게 제공하는데 동의하십니까?동의 미동의□ 개인 및 사업장 정보 제3자 제공내역제공하는 자제공받는 자제 공 항 목제 공 목 적보 유 기 간근로복지공단광주광역시,광주경제진흥상생일자리재단(수행기관)광주광역시가 제공한 사업장의 근로자의 개인별 사회보험료 산출․부과·납부내역과 사회보험료 지원내역광주광역시 사회보험료 신청(지원) 사업장의 사회보험료 사업주 부담금 확인 및 사회보험료 지원금 산출에 활용해당 기업에 대한 광주광역시의사회보험료 지원 사업 지원 종료 시 까지□ 위와 같이 개인정보를 수집·이용 하고 제3자에게 제공하는데 모두 동의하십니까? 동의 미동의2025년 월 일사업장명 근로자 성명 (서명 또는 인)광주광역시 ․ 광주경제진흥상생일자리재단·근로복지공단 귀중서식4 중소기업 지원사업 통합관리시스템기업(신용)정보 수집·이용·제공 동의서❶ 수집·이용에 관한 사항수집·이용목적 중소기업시책에 참여하는 기업의 지원효과 분석 중소기업 지원사업의 효율화(만족도 조사)수집·이용항목 중소기업기본법 제20조의2에 따른 기업 소재지, 업종, 매출액, 납입자본금, 자산총액, 부채총액, 영업이익, 당기순이익, 개업일·휴업일·폐업일, 전자계산서 발급액, 전자지급 거래액, 일반연구ˑ인력개발비, 현금영수증 결제금액, 신용카드 결제금액, 신고한 수출 물품의 품명, 품목 번호, 총 신고가격, 목적지, 신고일 중소기업 지원사업에 대한 만족도 조사 목적의 지원이력 정보(전화번호, 이메일 등)수집·이용기관 · 중소벤처기업부수집·이용기간 · 중소기업 지원사업 참여 기준 이전 3개년부터 사업 참여 이후 5년까지❷ 기업(신용)정보의 파기파기대상 정보 · 국세청 및 관세청 과세정보 · 개인을 식별할 수 있는 전화번호, 이메일 등파기절차 및 방법 · 파기 계획을 수립하여, 수집 및 이용기간 경과 시 30일 이내 지체 없이 파기 · 복원이 불가능한 방법으로 영구 삭제 본인은 위 목적으로 동의서에 적시된 정보 및 기관에 한해 기업정보를 수집‧이용‧제공하는 것에 동의합니다.※본동의서징구는민법제114조의행정행위의대리권행사방식의현명주의를적용하여,중소기업지원사업의효율적운영을위해본사업의소관부처및수행기관이대신하여받는것임을알려드립니다.※귀하는기업정보제공에대한동의를거부할권리가있습니다.동의를거부할경우지원사업맞춤형서비스이용에제한을받을수있습니다. 2025년월일 기업명ㅇㅇㅇ 대표자ㅇㅇㅇ(인)'},
        {'role' : 'assistant', 'content' : """
        📌 [광주] 2025년 1인 자영업자 및 소상공인 사회보험료 지원사업 요약 안내
         
        🤖 사업 개요
        - 경기침체로 어려움을 겪는 소상공인의 경영난 해소와 근로자 고용안정망 구축을 위하여 ‘1인 자영업자 및 소상공인 사회보험료 지원사업’의 참여자를 다음과 같이 모집 공고합니다.

        ✅ 사업자 유형
        - 개인사업자
        - 기존사업자
         
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 소상공인
        - 두루누리지원금
         
        👥 지원대상
        - (1인 자영업자) 고용 및 산재보험법에 따라 기(신규) 가입한 1인 자영업자
        - (소상공인) 정부 두루누리 사업장으로 관내 10인 미만 근로자를 고용한 사업장의 사업주

        📅 신청기간
        - 25. 1. 1. ~ 사업비 소진 시까지

        📝 신청방법
        - 1) 1인 자영업자 사회보험료 지원사업 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청
        - 온라인 신청 : gjbizinfo.or.kr (광주기업지원시스템)
        * 최초 1회 신청, 사업자 변경(사업주, 사업자등록번호 등)되는 경우 변경신청 필수
        * 기신청했던 업체에서는 신청자 변동 없을 시 별도 신청 없이 지급 예정
        - 오프라인 신청 : 방문 접수 (경제진흥상생일자리재단 3층 소상공인디지털전환실) 
        2) 소상공인 사회보험료 지원사업 : 온라인 신청 원칙, 온라인 신청 어려울 시 오프라인 신청
        - 온라인 신청 : gjbizinfo.or.kr (광주기업지원시스템), 담당자 이메일 접수
        * 기존에 신청했던 업체에서는 신청자 변동 없을 시 별도 신청없이 지급 예정
        - 오프라인 신청 : 방문 접수 (경제진흥상생일자리재단 3층 소상공인디지털전환실)
         
        🚫 지원제외 대상
        - (소상공인) 업주 본인, 건설·벌목업, 예술인, 특수형태 근로종사자 사업장, 사업주의 배우자 및 직계존비속 근로자의 보험료 미지원
         
        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X
         
        💁 청년 대상
        X
         
        ⚠ 유의사항
        1. 두루누리 사회보험 지원금을 받은 월에 한하여 광주광역시 사회보험료를 지원하며, 두루누리 
        사회보험 지원금은 해당 월 보험료를 법정 납부기한까지 완납하여야만 지원됩니다.
        ※ 법정 납부기한 후에 보험료를 완납하여도 두루누리 사회보험 지원금은 지원되지 않습니다.
        2. 사회보험료가 체납된 사업장은 사회보험료 지원을 받을 수 없습니다.
        3. 신청 사업장의 신규 입사자가 사회보험료 지원 최초 신청 시 개인정보 수집‧이용 및 제3자 제공
        동의서를 제출해야 하며, 퇴사자에 대한 지원 신청 시 퇴사자도 개인정보 수집‧이용 및 제3자 
        제공동의서를 제출해야 지원이 가능합니다.
         
        📑 제출서류    
        1인 자영업자 사회보험료 지원사업 제출서류
        | 구분          | 제출서류                                                                 | 비고                                                                 |
        |---------------|--------------------------------------------------------------------------|----------------------------------------------------------------------|
        | 필수 제출서류 | ① 사업신청서(행정정보공동이용동의서 포함) [서식1]                      |                                                                      |
        |               | ② 중소기업통합관리시스템 기업정보수집·이용·제공동의서 [서식4]          |                                                                      |
        |               | ③ 통장사본(스마트폰 캡처본도 인정)                                      | 수수료 없음                                                          |
        | 선택 제출서류 | ① 사업자등록증 또는 사업자등록증명원 사본 1부                          |                                                                      |
        |               | ② 건강보험자격득실확인서 사본 1부 (신청일로부터 1개월 이내 발급본)     | * 국민건강보험공단 홈페이지(www.nhis.go.kr) 또는 정부24에서 발급 가능 |
        |               |                                                                          | 행정정보공동이용 미동의 시 별도 제출               

         소상공인 사회보험료 지원사업 제출서류

        | 구분          | 신청주기 | 제출서류                                                                 | 비고 |
        |---------------|----------|--------------------------------------------------------------------------|------|
        | 신규 신청    | 최초 1회 | ① 소상공인 고용·산재보험료 지원신청서 [서식2]                           |      |
        |               |          | ② 사회보험료 지원 체크리스트 [서식3]                                    |      |
        |               |          | ③ 중소기업통합관리시스템 기업정보수집·이용·제공동의서 [서식4]          |      |
        |               |          | ④ 개인정보수집·활용동의서(근로자용) [첨부] / 보험료 지원신청 근로자 전체 |      |
        |               |          | ⑤ 사업자등록증, 사업자등록증명원, 고유번호증(비영리단체) 사본          |      |
        |               |          | ⑥ 사업주 또는 사업장 명의 통장사본(스마트폰 캡처본도 인정)             |      |
        |               |          | ⑦ 4대보험 사업장 가입자 명부                                           |      |
        | 변경 신청    | 근로자 추가 시 (추가 신청자만 해당) | ① 소상공인 고용·산재보험료 지원신청서 [서식2]                           |      |
        |               |          | ② 사회보험료 지원 체크리스트 [서식3]                                    |      |
        |               |          | ③ 개인정보수집·활용동의서(근로자용) [첨부] / 변경신청 근로자 전체      |      |
         
        📌 선정절차
        - (1인 자영업자) 고용・산재보험 가입 및 납부 ➡ 온·오프라인 신청(기업지원시스템·이메일 접수, 방문접수) ➡ 고용·산재보험 납부내역 확인 및 지급대상 검토 ➡ 사실관계 확인 후 보험료 지원금 지급
        - (소상공인) 두루누리 사회보험 지원신청 ➡ 보험료 先납부 서류접수(온·오프라인) ➡ 고용·산재보험 납부내역 확인 및 지급대상 검토 ➡ 보험료 지원금 지급
         
        💰 지원내용
        - (1인 자영업자) 부담한 고용보험료 부과액의 20%, 산재보험료 부과액의 50%
        - (소상공인) 근로자의 고용산재 보험료 중 정부지원을 제외한 사업주 부담분(고용보험 20%, 산재보험 100%) 지원
         
        📞 문의처
        - 1인 자영업자 사회보험료 지원사업 : 
        | 구분                       | 담당기관                     | 전화번호   |
        |:---------------------------|:-----------------------------|:-----------|
        | 고용·산재보험 광주광역시 지원문의 | (재)경제진흥상생일자리재단 소상공인디지털전환실 | 062-960-2632 |
        | 고용·산재보험 신규가입 문의       | 근로복지공단                   | 1588-0075  |
        | 고용보험 정부지원 문의           | 소상공인시장진흥공단             | 1357       |
        - 소상공인 사회보험료 지원사업 :
        | 구분                       | 담당기관                     | 전화번호   |
        | 사회보험료 광주광역시 지원문의     | (재)경제진흥상생일자리재단 소상공인디지털전환실 | 062-960-2632 |
        | 두루누리 사회보험료 지원문의       | 근로복지공단                   | 1588-0075  |
        | 국민연금공단                 | 근로복지공단                   | 1355       |
        """},
        ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '- 16 -□ 창업기반지원자금(청년전용창업자금) ◦ (대상) 대표자가만39세이하로서, 업력3년미만인중소기업 또는중소기업을창업하는자 * 단, 창업성공패키지사업(청년창업사관학교, 글로벌창업사관학교) 참여 기업, 청년창업기업보증 지원 기업(기보) 및 민간VC 투자 유치 기업 중 해당 기관의 추천을 받은 우수 기업은 업력 7년 미만까지 가능 ** 청년전용창업자금 대상 중 중점지원분야(참고1~4)를 영위하는 소상공인은 청년전용창업자금 신청 가능 ◦ (융자방식) 직접대출 ◦ (대출조건)구분대출 조건대출한도기업당 최대 1억원 이내 (제조업 및 중점지원분야 영위기업은 2억원 이내)대출기간(직접대출, 시설) 10년 이내 (거치기간 : 담보 4년 이내, 신용 3년 이내)(직접대출, 운전) 6년 이내 (거치기간 : 3년 이내)대출금리2.5% (고정금리) 기 타자금신청‧접수 후, 사업계획서 등 심의위원회 평가를 통해지원 여부 결정 후 대출※ 성장공유형 대출 및 투자조건부 융자의 세부 지원대상, 지원조건, 지원절차 등은 추후 공고'},
        {'role' : 'assistant', 'content' : """
        📌 혁신창업사업화자금-창업기반지원자금(청년전용창업자금)(2025년 중소기업 정책자금 융자계획 공고) 요약 안내
            
        🤖 사업 개요
        - 

        ✅ 사업자 유형
        - 법인사업자
        - 기존사업자
            
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 소상공인
        - 청년대상
        - 대출
        - 창업패키지
            
        👥 지원대상
        ㅇ 신청대상 : 대표자가 만 39세 이하로서 , 업력 3년 미만인 중소기업 또는 중소기업을 창업하는 자
        * 단, 창업성공패키지사업(청년창업사관학교, 글로벌창업사관학교) 참여 기업, 청년창업기업보증 지원 기업(기보) 및 민간VC 투자 유치 기업 중 해당 기관의 추천을 받은 우수 기업은 업력 7년 미만까지 가능
        ** 청년전용창업자금 대상 중 중점지원분야를 영위하는 소상공인은 청년전용창업자금 신청 가능

        📅 신청기간
        자금 소진시까지

        📝 신청방법
        - 
            
        🚫 지원제외 대상
        -
            
        ⭐️ 우대사항
        - 
            
        👩 여성 대상
        X
            
        💁 청년 대상
        O
            
        ⚠ 유의사항
        - 
         
        📑 제출서류
            
        📌 선정절차
        - 
            
        💰 지원내용
        | 구분 | 대출 조건 | 대출 한도 |
        |------|-----------|------------|
        | 직접대출, 시설 | 10년 이내 (거치기간 : 담보 4년 이내, 신용 3년 이내) | 기업당 최대 1억원 이내 (제조업 및 중점지원분야 영위기업은 2억원 이내) |
        | 직접대출, 운전 | 6년 이내 (거치기간 : 3년 이내) | 기업당 최대 1억원 이내 (제조업 및 중점지원분야 영위기업은 2억원 이내) |
            
        📞 문의처
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '전라남도 공고 제2025 - 8호2025년도 전라남도 중소기업 육성자금 지원계획 공고 전라남도 중소기업 육성기금 조례제12조의 규정에 따라 2025년도 전라남도 중소기업 육성자금 지원계획을 다음과 같이 공고합니다.2025. 1. 2.전라남도지사5 중소유통업 구조개선자금 지원 1. 지원규모 : 30억원 은행상담 후 자금 신청을 하여야 하며, 은행 여신규정에 의한 담보(보증서 등) 제공이 있어야 대출을 받을 수 있음. 2. 지원대상 지원대상 업종 - 지원 신청일 현재 도내에 사업자등록을 한 업체 한국표준산업분류 중 도소매업 및 상품중개업자 프랜차이즈 본부 및 가맹점(유통업에 한함) - 전통시장, 중소백화점쇼핑센터 동의 개설자 또는 관리자지원 제외 대상 국세지방세 체납중인 기업 휴폐업중인 업체 전라남도 중소기업 육성자금(시설자금) 이용 중이거나, 당해연도 추천 후 대출하지 않은 기업(단, 이용한도가 남은 기업은 한도 내 이용 가능) 당해 연도 정부 및 공공기관으로부터 시설자금 수혜기업 중소벤처기업부 중소기업지원사업 통합관리시스템에서 접수일 기준 최근 5년간 정부기관 및 지자체 등의 지원금액(사업분야 : 금융)과 본 신청 건을 포함하여 지원금액이 100억원을 초과하는 기업 금융 보증/보험 제외 기타 자금지원 관련 규정에서 제외하도록 정한 업체연속지원 제한 시설자금 상환 후 2년 이내, 운영자금 상환 후 1년 이내 융자 지원 제한 3. 지원내용구 분지원사업융자금액융자기간융자이율시설투자 전문상가 건립 전문상가 시설개선 공동창고 건립4억원 이내3년거치 5년 분할상환연3.0%(25.1분기 기준)- 정부정책자금 기준금리에 따라 분기별 변동 금리 적용 점포시설개선1억원 이내운영자금 경쟁력강화 및 경영활성화 자금1억원 이내1년거치 2년 분할상환 4. 지원신청 : 반기별 접수(공고일 ~, 6. 1. ~) 5. 접수방법 : 온라인 접수 - 전남중소기업일자리경제진흥원 자금시스템(http://www.jnfund.krh) - 문 의 처 : 061-288-3832~4 (기업육성팀) 6. 구비서류 공통서류 가. 자금신청서(별지 제1호), 사업계획서(별지 제2호), 정보이용동의서(별지 제4호) 각 1부(원본) 법인인 경우 반드시 법인도장 날인, 개인사업자 중 공동대표의 경우 대표자 전원 도장 날인 나. 대출상담확인서(별지 제3호) 1부(원본) 다. 사업자등록증 1부(사본) 라. 사업장 확인서류 1부(사본) - 자가 : 건축물대장(소유자 확인 필요) - 임대 : 임대계약서 + 건축물대장(소유자 확인 필요) 법인인 경우, 사업장 소유자가 대표자일 때 법인과 대표자간 임대차계약서 마. 세무사 또는 공인회계사가 확인한 최근 1개년 재무제표(또는 표준재무제표증명원-국세청 홈택스 발행분) 1부 결산이 완료되기 전인 1~3월경까지는 법인도 전년도분에 한해 부가가치세과세표준증명원으로 대체 가능 사업별 추가 구비서류 가. 전문상가 시설개선사업 - 전문상가 관리 법인 또는 입점상인 단체를 증명하는 서류 1부 - 입점상인의 시설 개보수 동의서 - 개보수 시공업체 사업자등록증, 계약서, 견적서 1부 나. 전문상가 및 공동창고 건립사업 - 전문상가 관리 법인 또는 입점상인 단체를 증명하는 서류 1부 - 토지등기부등본, 건축허가서 1부 - 시공업체 사업자등록증 사본, 건축시공 계약서 사본, 설계내역서, 건축도면 각 1부 다. 점포시설개선사업 : 개보수시공업체 사업자등록증, 계약서, 견적서 각 1부 라. 운영자금 : 매입세금계산서/합계표(신청금액 상당액, 회계사 또는 세무사 확인 표지 포함), 매입처 사업자등록증(신청금액 상당액 해당업체) 사본 1부 7. 지원업체 선정 전라남도 중소기업 육성기금 조례 시행규칙제13조의 규정에 따라 제외 대상업체 및 결격 여부를 심사하여 융자대상 업체 선정 8. 대출기한 : 추천서 발급일로부터 3개월 대출기한 내에 기업체가 기한연장을 신청할 경우 1회에 한하여 2개월 연장 가능(대출 진행 중이거나 공장시설 공사가 진행 중인 업체로 필요하다고 인정되는 경우) 9. 대출 취급은행 취급은행(10개) : 광주, 기업, 산업, 국민, 하나, 신한, 우리, 농협중앙회, 수협, 부산 기금관리은행(광주은행)과 전대 약정 미체결 은행은 취급은행에서 제외 10. 제재 및 환수 사항 지원 용도 외 자금 사용(임대수입, 용도전환, 후속계획 미이행 등) 기업은 해당자금에 대하여 전액 회수 및 향후 3년간 융자 추천 제한 거짓이나 부정한 방법으로 융자금을 (신청)지급 받은 경우 신청지원 목적과 다른 용도에 융자금을 사용한 경우(사업장 매매임대 등 수익 향유, 특수관계인 간 계약 등 거래, 업종 변경내역 미신고 등) 사업완료 후 1개월 이내 기업별 사업 결과보고서 제출(별지 2-1)을 의무화하고, 미제출 기업에는 당해연도와 다음연도 재신청(지원) 제한 11. 기타사항 공고 내용 이외의 사항은 전라남도 중소기업 육성기금 조례및 같은 조례 시행규칙, 2025년도 전라남도 중소기업 육성자금 지원계획에 의하며, 접수된 서류는 반환하지 않음 접수 후 심사 과정에서 필요시 추가 자료를 요청할 수 있음. 추천 및 대출 실행 후 거래은행, 상호 및 대표자 등 변경사항 발생 시 변경신청서(별지 제5호 서식)를 작성하여 전라남도중소기업일자리경제진흥원에 제출하여야 함. 신청 서식은 전남도청, 전남중소기업일자리경제진흥원 누리집에서 확인 가능 전남도청(http://www.jeonnam.go.kr)h : 분야별 정보 일자리경제 기업지원 자금지원 전남중소기업일자리경제진흥원(http://www.jepa.kr) : 자금지원 자금지원 공고 전남중소기업일자리경제진흥원 자금시스템(http://www.jnfund.kr)h : 정보광장 공지사항 온라인 접수 후 심사과정에서 필요한 추가요청자료를 1개월 이내에 보완하지 못할 시 신청이 취소됨 기타 자세한 사항은 전남중소기업일자리경제진흥원(061-288-3831~3)로 문의하시기 바람.[별지 제1호 서식] 전라남도 중소유통업 구조개선자금 신청서(시설 운영)필수사항업 체 명개 업년월일대표자성 명사 업 자등록번호생년월일 남 여법 인등록번호전 화 사업장소재지 자가 임차주소 : ( - ) 팩 스 핸 드 폰E-mail 업 종 제조 제조서비스 지식서비스 기타( )주 품 목(종 목) 업체구분 일 반 기 업 전 문 상 가 ( 중소백화점, 쇼핑센터 전통시장) 기타( )담 보제 공방 법 부동산 보증서 신용보증재단 신용보증기금 기술보증기금 기 타 ( )대 출은 행 은행 ( 지점)신청액계백만원시설자금백만원목적외 사용금지 목적외 사용시 융자금을 조기 환수, 지원 제한 등 제재조치가 취해질 수 있음을 확인함운영자금백만원준필수사항종업원수(상시근로자)담 당 자연 락 처전 화 : 팩 스 : 핸드폰 :E-mail : 매 출 액(최근1년) 백만원총 소요자금 백만원지방중소기업육성자금 기대출잔액 백만원 위와 같이 전라남도 중소유통업 구조개선자금을 신청하며 기재사항 및 붙임서류에 허위가 없음을 확인합니다. 2025년 월 일 신청인(대표) (인) 전라남도지사전라남도중소기업일자리경제진흥원장 귀하[별지 제2호 서식]사 업 계 획 서 사업기간 : 년 월 - 년 월 사업개요 위치, 규모(부지, 건물연면적), 구조, 용도 등 기재 소요자금 및 조달계획주2) (단위:백만원)구 분총 소 요 자 금유통업 구조개선자금 (신 청 자 금)기 타 자 금물량(면적/수량)금액물량(면적/수량)금액타 차입금자기자금총 계시설자금소 계전문상가/공동창고건 축기계설비점포시설개선운 영 자 금 주2) 시설자금 목적별(건축/기계설비)로 대출은행과 협의 후 신청자금 기재 융자금 상환계획 기 타 [별지 제2-1호 서식]중소기업육성자금 사업완료 보고서1. 기업현황업 체 명대 표 자소 재 지전화번호주생산품종업원수 명(남 명, 여 명)2. 지원현황 20 년도 제 차 자금 가. 시 설시 설 명규 격수 량시공업체(전화번호)사업기간 나. 융자금대출은행결정 통보액대출금액대출완료일자 은행 지점 계 : 백만원 시설: 백만원 운전: 백만원 계 : 백만원 시설: 백만원 운전: 백만원 3. 사업추진 중 문제점 및 건의사항 붙임 : 1. 사업추진 사진(사업별 전, 후 각 1매) 2. 시설자금(대출금액)의 세금계산서 사본 위와 같이 사업완료 보고서를 제출합니다. 년 월 일 기업명 : 대표자 : (인)전라남도지사전라남도중소기업일자리경제진흥원장 귀하[별지 제3호 서식]대 출 상 담 확 인 서 업체명 : 대표자 : 대출가능(상담)금액 : 원 위 업체는 금번 귀 도의 중소기업 육성자금을 지원받기 위하여 당행과 사전에 상담을 하였으므로, 귀 도에서 자금지원을 결정할 경우 지원 결정된 금액 이내로 당행 내규 등에 위배되지 않는 범위 내에서 대출 취급할 수 있음을 확인합니다.(단, 확인서 유효기간은 접수일 기준 1개월 이내 작성분에 한함)2025년 월 일담당자 : 은행 지점 (인)확인자 : 은행 지점장 (인) 전라남도지사전라남도중소기업일자리경제진흥원장 귀하 [별지 제4호 서식]기업정보의 제공활용 및 이용 동의서전라남도지사전라남도중소기업일자리경제진흥원장 귀하 본인은 전라남도 중소유통업구조개선자금 신청서및 제반서류(사업계획서, 기타 자금 융자에 필요한 서류 일체)에 기재된 기업신용정보를 해당 시군 및 공공기관의 정책 자료로 활용하도록 하는 것에 대해 동의합니다. 개인정보보호법에 따라, 전라남도 중소기업 육성기금 조례등에 따른 지원업체로 결정될 경우 당사의 정보(업체명, 대표자명, 사업자번호, 지원결정일, 지원사업명, 지원금액 등)를 중소기업 지원사업 통합관리시스템에 공개하는 것에 동의합니다. 중소기업 지원사업 통합관리시스템에서 사업 참여(신청 및 지원) 기업정보의 수집조회 및 활용 중소기업기본법 제20조의2에 따라 중소기업 지원사업에 신청한 기업의 과세정보(국세기본법 제81조의13 및 관세법 제116조), 고용정보(고용보험법 제2조의1), 직장가입자 수(국민건강보험법 제6조) 등 기업정보는 신청 및 지원기업의 사업 신청 이전 3개년부터 참여 이후 5년간 수집 중소기업 지원사업 통합관리시스템에 대한 동의서 징구는 민법 제114조의 행정행위의 대리권 행사방식의 현명주의를 적용하여, 중소기업 지원사업의 효율적인 운영을 위해 본 사업의 소관부처 및 수행기관이 대신하여 받는 것임을 알려 드립니다. 전라남도에 자금을 융자받기 위해 제출하는 제반서류(신청서, 사업계획서, 기타 자금 융자에 필요한 서류 일체) 제출 시 사실과 다르거나 허위의 자료를 제출 하여 융자를 신청한 경우에는 자금지원 결정취소, 조기상환, 신규참여제한 등의 제재 또는 법적조치가 취해질 수 있다는 것에 동의합니다.2025년 월 일 기 업 명 : 대 표 자 : (인)* 본인은 위 각 정보의 수집조회활용에 관한 자세한 설명을 듣고, 본 동의서 내용을 충분히 이해하고 동의하였습니다.[별지 제5호 서식] 자금추천서 발급 후 해당기업만 제출(대출취급일자 연장 또는 은행변경 시 작성전라남도 중소기업소상공인 육성자금 변경 신청(신고)서지원자금명 년 제( )차 ( ) 자 금업 체 명(사업자등록번호)( - - )대표자명(생년월일)( )소 재 지연 락 처전화)팩스)H P)변 경 사 유(상세하게 기재) 은행변경 및 대출기한 연장 등 해당기업만 작성 제출변 경 전변 경 후 상기와 같이 전라남도 중소기업 육성자금을 변경 신청합니다. 2025년 월 일 신청인 (인) 전라남도지사 전라남도중소기업일자리경제진흥원장 귀하 붙임서류 은행(지점) 또는 대출취급기한 변경시 - 육성자금추천결정통보서 사본 1부 대표자 또는 주소변경시 - 사업자등록증 및 법인등기부등본(말소사항포함) 각 1부신고대상 : 대표자 변경사항, 상호소재지의 변경, 휴폐업, 담보 종류의 변경, 기타 업체의 변동사항승인대상 : 법인 전환, 주생산업종의 변경, 계약대상시설의 변경, 추천기한취급은행 변경 문의 : 061)288-3832 / 접수 : www.jnfund.krh'},
        {'role' : 'assistant', 'content' : """
        📌 '[전남] 2025년 중소기업 육성자금(중소유통업 구조개선자금) 지원계획 공고'
         
        🤖 사업 개요
        - 「광주광역시 중소기업육성기금 특별회계설치 및 운용조례」 제7조의 규정에 따라 2025년도 광주광역시  중소유통업구조개선자금 지원계획을 다음과 같이 공고합니다.

        ✅ 사업자 유형
        - 법인사업자
        - 개인사업자
         
        ☑️ 지원업종
        - 도매 및 상품 중개업
        - 소매업(자동차 제외)

        💡 공고특성
        - 소상공인
        - 대출
        - 보조금
        - 시설·환경 개선
         
        👥 지원대상
        - 지원 신청일 현재 전라남도 내에 사업자등록을 한 업체
        - 한국표준산업분류 중 도소매업 및 상품중개업자
        - 프랜차이즈 본부 및 가맹점(유통업에 한함)
        - 전통시장, 중소백화점쇼핑센터 동의 개설자 또는 관리자

        📅 신청기간
        반기별 접수(공고일 ~, 6. 1. ~)

        📝 신청방법
        - 온라인 접수 
        - 전남중소기업일자리경제진흥원 자금시스템(www.jnfund.kr) 
        ㅇ 문 의 처 : 전남중소기업일자리경제진흥원 기업육성팀 (061-288-3832~4)
         
        🚫 지원제외 대상
        - 국세지방세 체납중인 기업
        - 휴폐업중인 업체
        - 전라남도 중소기업 육성자금(시설자금) 이용 중이거나, 당해연도 추천 후 대출하지 않은 기업(단, 이용한도가 남은 기업은 한도 내 이용 가능)
        - 당해 연도 정부 및 공공기관으로부터 시설자금 수혜기업
        - 중소벤처기업부 중소기업지원사업 통합관리시스템에서 접수일 기준 최근 5년간 정부기관 및 지자체 등의 지원금액(사업분야: 금융)과 본 신청 건을 포함하여 지원금액이 100억원을 초과하는 기업(금융 보증/보험 제외)
        - 기타 자금지원 관련 규정에서 제외하도록 정한 업체

        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X
         
        💁 청년 대상
        X
        
        ⚠ 유의사항
        - 지원 용도 외 자금 사용(임대수입, 용도전환, 후속계획 미이행 등) 기업은 해당자금에 대하여 전액 회수 및 향후 3년간 융자 추천 제한
        - 거짓이나 부정한 방법으로 융자금을 (신청)지급 받은 경우
        - 신청지원 목적과 다른 용도에 융자금을 사용한 경우(사업장 매매임대 등 수익 향유, 특수관계인 간 계약 등 거래, 업종 변경내역 미신고 등)
        - 사업완료 후 1개월 이내 기업별 사업 결과보고서 제출(별지 2-1)을 의무화하고, 미제출 기업에는 당해연도와 다음연도 재신청(지원) 제한
         
        📌 선정절차
        - 전라남도 중소기업 육성기금 조례 시행규칙제13조의 규정에 따라 제외 대상업체 및 결격 여부를 심사하여 융자대상 업체 선정
         
        💰 지원내용
        - 지원규모 : 30억원
        | 구분 | 지원사업 | 융자금액 | 융자기간 | 융자이율 |
        |------|----------|----------|----------|----------|
        | 시설 투자 | 전문상가 건립, 전문상가 시설개선, 공동창고 건립 | 4억원 이내 | 3년거치 5년 분할상환 | 연3.0% (25.1분기 기준) - 정부정책자금 기준금리에 따라 분기별 변동 금리 적용 |
        | 점포시설개선 | | 1억원 이내 | | |
        | 운영 자금 | 경쟁력강화 및 경영활성화 자금 | 1억원 이내 | 1년거치 2년 분할상환 | |
        <연속지원 제한>
        - 시설자금 상환 후 2년 이내, 운영자금 상환 후 1년 이내 융자 지원 제한
         
        📞 문의처
        061-288-3832~4 (기업육성팀)
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : """
        당신은 정부 지원 사업 전문 분석가입니다. 
        사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 추출해야합니다. 파일 내용을 철저하게 분석하고, 어떤 항목도 누락시키지 마세요. 파일에 없는 내용은 공백으로 남겨두십시오.
        요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니 사전에 해당하는 단어가 나오면 참고해주십시오.

        사업자 유형, 지원업종, 공고특성, 우대사항 항목은 문맥을 분석해 가장 적합한 단어를 단어사전 내에서 선택해주세요. 조건에 해당하는 사항이 중복되는 경우, 각 대상에 모두 포함시켜주세요.
        다른 항목의 경우는 사전에 해당하지 않는 단어가 나오면, 해당 단어를 그대로 사용하여 요약해주십시오.

        신청기간과 제출서류는 각 항목을 세분화할 수 있는 경우, 세분화하여 요약해주세요.

        지원대상, 신청방법은 모든 내용을 누락없이 추출해주세요. 접수처와 접수처 현황까지 모두 포함해야 합니다.
        신청기간과 제출서류가 자금마다 두 가지 이상으로 다르게 제시된 경우에만 '공고문 참조'로 대체해주세요. 만약 신청기간이나 제출서류가 하나로 통일되어 있거나, 하나의 기간 혹은 제출서류만 존재하는 경우에는 구체적인 값을 출력해주세요.

        신청방법, 지원내용 항목의 경우 표로 제시된 정보는 그대로 표 형태로 출력해주세요. 표는 데이터프레임 형식으로 출력해주세요.
                
        제공하는 템플릿 양식 : {base_template}
        단어 정의 사전 : {keyword_dictionary}
        참고 : {explanation}

        각 항목의 범위를 명확히 구분하고, 잘못 분류된 정보를 수정해주세요.
        """},
        {'role' : 'user', 'content' : '2025년 상반기 보증연계투자 대상기업 공모기술보증기금(이하 기보)에서는 기술력과 사업성이 우수한 유망 창업기업에 대한 투자 활성화를 위해 다음과 같이 2025년 상반기 보증연계투자 대상기업을 공모하니, 많은 관심과 참여를 바랍니다.실시 목적 기술력과 사업성이 우수하고 미래 성장이 기대되는 유망 창업기업의 육성을 위해 선제적으로 공적 투자를 지원신청 자격 기술력, 사업성 및 성장잠재력 등을 갖춘 중소기업으로, 아래 기본요건을 모두 충족하는 창업기업법인설립 후 5년 이내의 기술혁신선도형기업* * 단, R&D 및 녹색혁신성장미래전략산업 영위기업 등은 설립 후 5년 초과7년 이내 기업도 가능기보 보증기업 또는 보증승인(예정)기업최근 1년 이내 기보 기술사업평가등급이 BB등급 이상인 기업* * 최근 1년 이내 평가등급이 없더라도 기보 영업점 담당자가 투자지원 가능성이 높다고 판단한 기업은 신청 가능보증금지기업 또는 보증제한기업에 해당되지 않는 기업벤처캐피탈, 은행 등 기관투자자로부터 투자를 받지 않은 기업* * 단, 지방소재기업 및 모태출자조합으로부터 3억원 이하 투자를 받은 기업은 신청 가능보증연계투자(투자옵션부보증 포함) 잔액이 없는 기업* 위 , , , 해당 여부는 기보 영업점 담당자에게 문의공모 절차 및 지원 내용 보증연계투자 대상기업 공모 절차 공모제시행공고공모추천예비검토기술평가,투자심사투자의사결정투자실행홈페이지 등상담/접수1차선별2차선별최종 결정계약체결 등벤처혁신금융부영업점*(콘텐츠)벤처투자금융센터(콘텐츠)벤처투자금융센터/기술혁신센터**(콘텐츠)벤처투자금융센터/투자심의회(콘텐츠)벤처투자금융센터 * 중앙기술평가원, 기술혁신센터, 소셜벤처가치평가센터, 서울문화콘텐츠금융센터, 지식재산공제센터 등 포함** 투자심사용 기술평가 업무수행 (공모기간) 25.3.17(월) 25.5.30(금) (접수방법) 기보 영업점 담당자와 상담 후 접수 (투자지원 규모) 25년 상반기 400억원 내외 (투자방식) 주식전환사채신주인수권부사채 인수, Convertible Note, SAFE 등 (투자한도) 개별 기업당 30억원 이내기타 사항 (유의사항) 예비검토 단계에서 기업 방문 또는 별도 통지 없이 투자 대상에서 제외될 수 있으며, 제출 서류는 반환하지 않음 (문의처) 관할 영업점 또는 신청기업 본사사업장 인근 영업점 * (고객센터)15441120, 전국 영업점 연락처는 홈페이지(www.kibo.or.kr) 참조투자일정절차, 공모대상 해당 여부, 기술평가등급 및 접수 방법 등 공모 관련 자세한 사항은 귀사의 관할 영업점으로 문의 바랍니다.'},
        {'role' : 'assistant', 'content' : """
        📌 2025년 상반기 보증연계투자 대상기업 공모 요약 안내
         
        🤖 사업 개요
        - 기술보증기금(이하 ‘기보’)에서는 기술력과 사업성이 우수한 유망 창업기업에 대한 투자 활성화를 위해 다음과 같이 2025년 상반기 보증연계투자 대상기업을 공모하니, 많은 관심과 참여를 바랍니다.

        ✅ 사업자 유형
        - 법인사업자
        - 개인사업자
         
        ☑️ 지원업종
        - 제조업, 정보통신업, 전문, 과학 및 기술 서비스업

        💡 공고특성
        - 대출
         
        👥 지원대상
        기술력, 사업성 및 성장잠재력 등을 갖춘 중소기업으로, 아래 기본요건을 모두 충족하는 창업기업
        - 법인설립 후 5년 이내의 기술혁신선도형기업
        * 단, R&D 및 녹색혁신성장미래전략산업 영위기업 등은 설립 후 5년 초과 7년 이내 기업도 가능
        - 기보 보증기업 또는 보증승인(예정)기업
        - 최근 1년 이내 기보 기술사업평가등급이 BB등급 이상인 기업
        * 최근 1년 이내 평가등급이 없더라도 기보 영업점 담당자가 투자지원 가능성이 높다고 판단한 기업은 신청 가능
        - 보증금지기업 또는 보증제한기업에 해당되지 않는 기업
        - 벤처캐피탈, 은행 등 기관투자자로부터 투자를 받지 않은 기업
        * 단, 지방소재기업 및 모태출자조합으로부터 3억원 이하 투자를 받은 기업은 신청 가능
        - 보증연계투자(투자옵션부보증 포함) 잔액이 없는 기업

        📅 신청기간
        25. 3. 17 ∼ 25. 5. 30 17:00까지

        📝 신청방법
        - 기보 영업점 담당자와 상담 후 접수
         
        🚫 지원제외 대상
        - 
         
        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X 
         
        💁 청년 대상
        X 
         
        ⚠ 유의사항
        - 
         
        📑 제출서류
        -

        📌 선정절차
        - 
         
        💰 지원내용
        - 
        
        📞 문의처
        - 
         
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '- 30 -4-2 구조개선전용자금□ (대상) 아래의유형(① ~ ③)에해당하는기업 ① 아래사항중하나에해당되어진로제시컨설팅(기초진로제시)을받은기업중‘구조개선’ 대상으로판정된기업• 정책금융기관 등(중진공, 신보, 기보, 테크노파크)이 지정한 부실징후기업• ｢채권은행협의회 운영협약｣ 또는 ｢기업구조조정 촉진법｣에 의한 워크아웃 추진기업으로 워크아웃계획을 정상 이행중인 경우• ｢채무자 회생 및 파산에 관한 법률｣에 따른 회생절차 종결 후 5년 이내 기업으로 법원에서 승인한 회생계획을 정상 이행중인 경우• 기타 경영애로를 겪고 있어 구조개선, 회생, 사업정리 등 향후 진로제시가 필요한 기업 * 진로제시컨설팅 시 기업 비용부담(컨설팅 비용의 10% 및 부가세 10%) 발생 ② 아래사항중하나에해당되어한국자산관리공사(캠코)에서금융지원을추천한‘캠코협업금융지원’ 대상기업•패키지형 회생기업 금융지원 참여기업 •국세 물납 법인•Sales & LeaseBack 참여(선정)기업 ③ ‘선제적자율구조개선프로그램’에서지원을결정한기업※ 선제적 자율구조개선 프로그램 개요• 중진공과 금융기관(협약은행 등)*이 협력하여 위기징후 중소기업을 선제 발굴하고, 공동 금융지원과 경영개선계획 수립 등으로 기업의 자율구조개선을 지원하여 신속한 경영정상화 촉진 및 재도약 기회 부여 * 기업은행, 수출입은행, 경남은행, 농협은행, 산업은행, 국민은행, iM뱅크, 신한은행, 우리은행, 하나은행, 부산은행, 기술보증기금, 지역신용보증재단 등 ** 지원절차:신청→상담·추천→사전검토→진단·평가및경영개선계획수립→지원결정→금융지원→이행점검 * 프로그램 세부내용 확인 : 중진공 누리집(www.kosmes.or.kr) 「지원사업 - 정책자금융자 - 세부사업 - 재도약지원자금 - 선제적 자율구조개선 프로그램」- 31 -※ 구조개선전용자금 우대 사항 • 융자계획 공고 Ⅲ. 유의사항(융자제한기업) ③항(세금체납기업), ④항(신용정보관리대상기업), ⑩항(부채비율초과기업), ⑪항(한계기업) 적용 예외(단, ③항(세금체납기업) 예외 적용의 경우는 압류·매각의 유예에 한하며, ④항(신용정보관리대상기업) 예외 적용의 경우는 ‘회생’ 정보 등록기업에 한함)□ (융자방식) 직접대출□ 대출조건구분대출 조건일반 구조개선전용자금(캠코 협업 금융지원 포함) 선제적 자율구조개선 프로그램대출한도시설- 연간 60억원 이내운전3년간 10억원 이내3년간 15억원 이내(연 10억원 이내)대출기간시설- 10년 이내(거치기간 : 담보 4년 이내, 신용 3년 이내)운전5년 이내 (거치기간 : 2년 이내)대출금리정책자금 기준금리(변동) 2.5%(고정)'},
        {'role' : 'assistant', 'content' : """
        📌 재도약지원자금-구조개선전용자금(2025년 중소기업 정책자금 융자계획 공고) 요약 안내
         
        🤖 사업 개요
        - 

        ✅ 사업자 유형
        - 법인사업자
        - 기존사업자
         
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 대출
         
        👥 지원대상
        아래의 유형(① ~ ③)에 해당하는 기업
        ① 아래 사항 중 하나에 해당되어 진로제시컨설팅(기초진로제시)을 받은 기업 중 ‘구조개선’ 대상으로 판정된 기업
        - 정책금융기관 등(중진공, 신보, 기보, 테크노파크)이 지정한 부실징후기업
        - ｢채권은행협의회 운영협약｣ 또는 ｢기업구조조정 촉진법｣에 의한 워크아웃 추진기업으로 워크아웃계획을 정상 이행 중인 경우
        - ｢채무자 회생 및 파산에 관한 법률｣에 따른 회생절차 종결 후 5년 이내 기업으로 법원에서 승인한 회생계획을 정상 이행 중인 경우
        - 기타 경영애로를 겪고 있어 구조개선, 회생, 사업정리 등 향후 진로제시가 필요한 기업
            * 진로제시컨설팅 시 기업 비용부담(컨설팅 비용의 10% 및 부가세 10%) 발생
        ② 아래 사항 중 하나에 해당되어 한국자산관리공사(캠코)에서 금융지원을 추천한 ‘캠코협업금융지원’ 대상기업
        - 패키지형 회생기업 금융지원 참여기업
        - 국세 물납 법인
        - Sales & LeaseBack 참여(선정)기업
        ③ ‘선제적자율구조개선프로그램’에서 지원을 결정한 기업
        ※ 선제적 자율구조개선 프로그램 개요
        - 중진공과 금융기관(협약은행 등)*이 협력하여 위기징후 중소기업을 선제 발굴하고, 공동 금융지원과 경영개선계획 수립 등으로 기업의 자율구조개선을 지원하여 신속한 경영정상화 촉진 및 재도약 기회 부여
            * 기업은행, 수출입은행, 경남은행, 농협은행, 산업은행, 국민은행, iM뱅크, 신한은행, 우리은행, 하나은행, 부산은행, 기술보증기금, 지역신용보증재단 등
            ** 지원절차 : 신청 → 상담·추천 → 사전검토 → 진단·평가 및 경영개선계획 수립 → 지원결정 → 금 융지원 → 이행점검

        📅 신청기간
        공고문 참조

        📝 신청방법
        - 
         
        🚫 지원제외 대상
        -
         
        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X
         
        💁 청년 대상
        X 
         
        ⚠ 유의사항
        - • 융자계획 공고 III. 유의사항(융자제한기업) ③항(세금체납기업), ④항(신용정보관리대상기업), ⑩항     (부채비율초과기업), ⑪항(한계기업) 적용 예외(단, ③항(세금체납기업) 예외 적용의 경우는 압류·매각의 유예에 한하며, ④항(신용정보관리대상기업) 예외 적용의 경우는 ‘회생’ 정보 등록기업에 한함)
        
        📑 제출서류
        -
         
        📌 선정절차
        - 
         
        💰 지원내용
        | 대출 조건 구분 | 대출 조건 |
        |----------------|-----------|
        | 일반 구조개선전용자금(캠코 협업 금융지원 포함) | |
        | 대출한도 | 시설- 연간 60억원 이내, 운전 3년간 10억원 이내 |
        | 대출기간 | 시설- 10년 이내(거치기간 : 담보 4년 이내, 신용 3년 이내), 운전 5년 이내 (거치기간 : 2년 이내) |
        | 대출금리 | 정책자금 기준금리(변동) 2.5%(고정) |
        
        📞 문의처

        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '- 35 -5-2긴급경영안정자금(일시적경영애로)□ (대상) ‘① 경영애로사유’로일정부분피해(‘② 경영애로규모’)를 입은일시적경영애로기업중경영정상화가능성이큰기업※ 긴급경영안정자금(일시적경영애로) 우대 사항 • 융자계획 공고 Ⅲ.유의사항(융자제한기업)⑩항(부채비율초과기업) 적용 예외• 정부 산업구조조정 대상 업종(조선, 자동차, 해운, 철강, 석유화학), 원전 협력 중소기업,대유위니아그룹 계열사 회생신청에 따른 피해 협력기업은 융자계획 공고 Ⅲ. 유의사항(융자제한기업) ①항(우량기업), ③항(세금체납기업), ⑩항(부채비율초과기업) 적용 예외(단, ③항(세금체납기업) 예외 적용의 경우에도 압류·매각의 유예에 한함) ① 경영애로사유•환율피해•대형사고(화재 등)•대기업 구조조정•정부의 산업구조조정 대상업종(조선, 자동차, 해운, 철강, 석유화학) 관련 피해•주요거래처 도산 및 결제조건 악화•기술유출 피해•불공정거래행위, 기술침해, 외국기업 또는 대기업과의 특허분쟁에 따른 피해•한‧중 FTA 지원업종(피해)(참고11)•고용위기 또는 산업위기지역 소재 중소기업•산업위기 선제대응지역 소재 중소기업•화학안전 법령 이행•중소기업 특별지원지역 경영애로•개성공단 입주 철수기업•우크라이나 사태 피해기업•원전 협력 중소기업(단, 사업계획 및 자금 사용 목적이 원전 사업 추진을 위한 경우에 한하여 지원)•원자재 가격급등으로 인한 생산 등의 애로 중소기업•정의로운전환 특별지구에 소재한 중소기업•대유위니아그룹 계열사 회생신청에 따른 피해 협력기업•위메프, 티몬, 인터파크쇼핑, AK몰 또는 알렛츠로부터 판매대금 미정산 피해를 입은 것으로 인정(셀러허브를 통한 입점기업 포함)되는 중소기업•기타 중소벤처기업부장관이 지원이 필요하다고 인정하는 사유 ※ 제품과 서비스의 시장성 부족 등에 따른 영업부진 등은 제외- 36 - ② 경영애로규모: 매출액또는영업이익이10% 이상감소한기업, 대형사고(화재등)로피해규모가1억원이상인기업비교시점 (연도) 직전연도와 직전전연도 (반기) 직전반기와 직전전반기, 직전반기와 전년동반기 (분기) 직전분기와 직전전분기, 직전분기와 전년동분기 (월별) 신청전월과 전전월, 신청전월과 전년동월※ 긴급경영안정자금(일시적경영애로) 경영애로 규모 요건 적용 예외•고용위기 또는 산업위기지역 소재 중소기업•조선업 관련 피해기업 •중소기업 특별지원지역 소재 중소기업•우크라이나 사태 피해기업 •원전 협력 중소기업•기술침해 피해기업 •대유위니아그룹 계열사 회생신청에 따른 피해 협력기업 ③ 신청기한: 경영애로피해발생(피해비교가능시점) 후6개월이내 * 단, 정부의 산업구조조정대상 업종 관련 피해기업은 신청기한을 경영애로 피해발생 후 1년 이내로 우대□ (융자방식) 직접대출□ 대출조건구분대출 조건대출한도(직접대출, 운전) 10억원 이내(3년간 15억원 이내)대출기간(직접대출, 운전) 5년 이내 (거치기간 : 2년 이내)대출금리정책자금 기준금리(변동) + 0.5%p기 타•운전자금의 용도는 경영애로 해소 및 경영정상화에 소요되는 경비'},
        {'role' : 'assistant', 'content' : """
        📌 긴급경영안정자금-일시적경영애로(2025년 중소기업 정책자금 융자계획 공고) 공고 요약
         
        🤖 사업 개요
        - 

        ✅ 사업자 유형
        - 법인사업자
        - 개인사업자
         
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 대출
         
        👥 지원대상
        - 경영애로사유로 일정 부분 피해를 입은 일시적 경영애로 기업 중 경영 정상화 가능성이 큰 기업

        📅 신청기간
        - 경영애로 피해 발생 후 6개월 이내 (정부의 산업구조조정대상 업종 관련 피해기업은 1년 이내) 

        📝 신청방법
        공고문 참조
         
        🚫 지원제외 대상
        - 고용위기 또는 산업위기지역 소재 중소기업
        - 조선업 관련 피해기업
        - 중소기업 특별지원지역 소재 중소기업
        - 우크라이나 사태 피해기업
        - 원전 협력 중소기업
        - 기술침해 피해기업
        - 대유위니아그룹 계열사 회생신청에 따른 피해 협력기업
         
        ⭐️ 우대사항
        - 융자계획 공고 Ⅲ. 유의사항(융자제한기업)⑩항(부채비율초과기업) 적용 예외
        - 정부 산업구조조정 대상 업종(조선, 자동차, 해운, 철강, 석유화학), 원전 협력 중소기업,대유위니아그룹 계열사 회생신청에 따른 피해 협력기업은 융자계획 공고 III. 유의사항(융자제한기업) ①항(우량기업), ③항(세금체납기업), ⑩항(부채비율초과기업) 적용 예외(단, ③항(세금체납기업) 예외 적용의 경우에도 압류·매각의 유예에 한함)
         
        👩 여성 대상
        X
         
        💁 청년 대상
        X 
         
        ⚠ 유의사항
        - 
        
        📑 제출서류
        공고문 참조
        
        📌 선정절차
        - 
         
        💰 지원내용
        | 구분 | 대출 조건 |
        |------|-----------|
        | 대출한도(직접대출, 운전) | 10억원 이내(3년간 15억원 이내) |
        | 대출기간(직접대출, 운전) | 5년 이내 (거치기간 : 2년 이내) |
        | 대출금리 | 정책자금 기준금리(변동) + 0.5%p |
        | 기타 | 운전자금의 용도는 경영애로 해소 및 경영정상화에 소요되는 경비 |
        
        📞 문의처
        공고문 참조
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '울산광역시 공고 제2025-5호2025년 울산광역시 식품진흥기금 융자사업 시행 공고울산광역시 식품진흥기금조례 제5조(융자사업)의 규정에 따라 2025년도 울산 광역시식품진흥기금 융자지원사업을 다음과 같이 공고합니다.2025.1.3.울 산 광 역 시 장1.융자지원 규모:5억원(식품진흥기금)2.융자조건:○ 연리 0.5%,2년 거치 3년 균등 분할 상환○ 신용,담보 등 경남은행 여신 규정 적합자3.융자대상:위생관리시설을 개선․확충하고자 하거나 영업에 필요한 기계,설비 등을 설치․보유하고자 하는 영업자4.융자 한도액○ 식품안전관리인증기준(HACCP)적용업소 :2억원 이내○ 식품제조․가공업소,즉석판매제조․가공업소 :1억원 이내○ 식품접객업소 :5천만원 이내(화장실만 개선하는 경우는 1천만원 이내)5.융자제외 대상○ 무신고 업소,신규업소(지위승계 포함)및 영업 신고 후 6개월 미만 업소○ 식품위생법 위반으로 영업정지(과징금 포함)에 해당하는 행정처분을 받고 1년이 경과 되지 아니하였거나,영업정지 이상의 행정처분이 진행 중에 있는 자 또는 휴·폐업 업소○ 식품접객업 중 단란주점영업 및 유흥주점영업(단,주방 및 화장실 개선은 가능)6.신청기간:연중(자금 소진 시까지)7.신청장소:영업장 소재지 구․군 위생부서8.구비서류:식품진흥기금 융자신청서 및 위생관리시설개선사업계획서9.융자 취급 금융기관:BNK경남은행 전 영업점10.융자금 환수대상○ 융자금을 목적 외의 용도로 사용한 경우○ 허위 또는 부정한 방법으로 융자를 받은 경우○ 융자받은 후 3개월 이내 위생관리시설 개선을 완료하지 아니한 경우 다만,시설개선을 완료하지 못할 불가피한 사유가 있어 관할 구청장․군수와 협의하여 기간을 연장한 경우는 제외○ 융자받은 업소가 폐업 또는 허가취소(폐쇄)된 경우○ 융자금에 대한 채무승계 없이 영업자 지위승계를 한 경우≪ 문 의 처 ≫▷ 시 식의약안전과 ☏ 229-3654▷ 중 구 환경위생과 ☏ 290-3732▷ 남구 위 생 과 ☏ 226-5722▷ 동 구 환경위생과 ☏ 209-3572▷ 북구 환경위생과 ☏ 241-7782▷ 울주군 위 생 과 ☏ 204-2212[붙임 1][별지제1호서식]<개정2017‧6‧29>식품진흥기금융자신청서신청인대표자 생년월일업소명(업종) 허가(신고)일자주소 사업자등록번호영업장소재지전화번호사업체:자택:사업체규모영업장총면적㎡연간매출액천원영업장소유구분자가,전세,월세,기타월평균소득액천원자금융자신청내역실소요금액(A)융자신청금액(B)비율(B/A)천원천원「울산광역시식품진흥기금운용조례」제5조제3항및같은조례시행규칙제4조제1항에따라식품진흥기금의융자를신청합니다.년월일 신청인(서명또는날인)구청장‧군수귀하※구비서류1.민원인제출서류:위생관리시설개선사업계획서1부.2.담당공무원확인사항:사업자등록증(다만,신청인이행정정보의공동이용에동의하지아니하는경우에는사업자등록증사본을첨부하여야한다)거래하는()은행○○지점(직인)(대출가능,불가능)확인자성명○○○(서명또는인)※거래희망은행에서대출심사과정중은행의여신규정에의거대출가능여부가변경될수있습니다.본인은이건업무처리와관련하여「전자정부법」제36조제1항에따른행정정보의공동이용을통하여담당공무원이사업자등록증을확인하는것에동의합니다. 신청인(서명또는인)210㎜×297㎜[일반용지 60g/㎡(재활용품)][붙임 2][별지제2호서식]<개정2010․5․13>위생관리시설개선사업계획서(제4조제1항관련)업소명소재지대표자생년월일영업장면적㎡○사업기간:○소요자금:천원(융자천원,자부담천원)○사업개요및시설개선내용-시설개선사항별소요금액등구체적으로기재◦◦◦◦◦○융자금상환계획○기타210㎜×297㎜[일반용지60g/㎡(재활용품)][붙임 3][별지제3호서식]<개정2010․5․13>융자신청현지조사서융자신청인성명 생년월일주소 전화번호업소현황업소명 허가(신고)번호업종 허가(신고)일자소재지 전화번호시설규모대지㎡건물규모㎡영업장면적㎡구조재산상태총액천원(동산천원,부동산천원)월평균소득액천원연간매출액천원영업장소유자가,전세,월세,기타()거래희망은행은행지점융자제외대상여부융자우선순위부여대상여부사업의타당성조사자의견「울산광역시식품진흥기금운용조례시행규칙」제4조제2항에따라식품진흥기금융자신청에대하여현지조사한결과를위와같이보고합니다. 년월일 조사자:소속직성명(서명또는날인)구청장‧군수귀하210㎜×297㎜[일반용지60g/㎡(재활용품)][붙임 4][별지 제4호서식]<개정 2010․ 5․ 13>행 정 기 관 명수신자 :울산광역시장(경유)제목 :식품진흥기금융자대상자 추천서「울산광역시식품진흥기금운용조례시행규칙」제4조제2항에따라식품진흥기금융자대상자를아래와같이추천합니다.붙임1.현지조사서(시설개선전사진포함)사본1부.2.융자신청서사본1부.3.사업계획서사본1부.4.영업허가(신고)대장사본1부.끝.(단위:천원)업종업소명소재지대표자융자신청내용융자대상우선순위시설개선계획소요금액신청금액울산광역시 구청장ㆍ군수직인기안자 직위(직급) 서명 검토자 직위(직급)서명 결재권자 직위 (직급)서명협조자시행 처리과-일련번호(시행일자)접수 처리과명-일련번호(접수일자)우주소 /홈페이지 주소전화( ) 전송( ) /기안자의 공식전자우편주소/공개구분210㎜×297㎜[일반용지 60ｇ/㎡(재활용품)][붙임 5][별지제5호서식]<개정2010․5․13>식품진흥기금융자업소관리대장(제7조제2항관련)업종 허가(신고)번호업소명 전화번호대표자 생년월일소재지융자금액 천원융자일자융자금리연리%융자기간상환방법사업내용융자금원금상환내역상환계획실제상환내역비고연월일금액잔액연월일금액잔액사후관리내역연월일내용담당자직‧성명210㎜×297㎜[일반용지60g/㎡(재활용품)][붙임 6]위생관리시설개선완료신고서신고번호업소명대표자신청자소재지전화번호융자금액신청일(구․군) 대출인(은행)융자및상환조건년거치년균등분활상환(연%)융자금의용도영업장등시설개선내역공사기간년월일~년월일까지(개월)개선시설명완공일자202년월일미착공시사유구체적으로기재착공예정일완공예정일시공업체명울산광역시식품진흥기금운용조례및시행규칙규정에의거위생관리시설개선완료신고서를제출합니다.년월일 대표자(인)울산광역시장귀하구청장,군수귀하첨부서류1.식품진흥기금융자금입금통장사본1부.2.위생관리시설개선완료사진각1부.(전,중,후)'},
        {'role' : 'assistant', 'content' : """
        📌 [울산] 2025년 식품진흥기금 융자사업 시행 공고 요약 안내
         
        🤖 사업 개요
        - 울산광역시 식품진흥기금조례 제5조(융자사업)의 규정에 따라 2025년도 울산 광역시식품진흥기금 융자지원사업을 다음과 같이 공고합니다.

        ✅ 사업자 유형
        - 개인사업자
        - 법인사업자
        - 기존사업자
         
        ☑️ 지원업종
        - 음식점업, 제조업

        💡 공고특성
        - 소상공인
        - 대출
         
        👥 지원대상
        - 위생관리시설을 개선·확충하고자 하거나 영업에 필요한 기계, 설비 등을 설치·보유하고자 하는 영업자

        📅 신청기간
        - 연중(자금 소진 시까지)

        📝 신청방법
        - 영업장 소재지 구․군 위생부서
         
        🚫 지원제외 대상
        - 무신고 업소, 신규업소(지위승계 포함) 및 영업 신고 후 6개월 미만 업소
        - 식품위생법 위반으로 영업정지(과징금 포함)에 해당하는 행정처분을 받고 1년이 경과 되지 아니하였거나, 영업정지 이상의 행정처분이 진행 중에 있는자 또는 휴·폐업 업소
        - 식품접객업 중 단란주점영업 및 유흥주점영업(단,주방 및 화장실 개선은 가능)
         
        ⭐️ 우대사항
        -
        
        👩 여성 대상
        X
         
        💁 청년 대상
        X
         
        ⚠ 유의사항
        -
        
        📑 제출서류
        - 식품진흥기금 융자신청서
        - 위생관리시설개선사업계획서
        
        📌 선정절차
        -
         
        💰 지원내용
        | 융자대상 | 융자 한도액 |
        |----------|-------------|
        | 식품안전관리인증기준(HACCP) 적용업소 | 2억원 이내 |
        | 식품제조·가공업소, 즉석판매제조·가공업소 | 1억원 이내 |
        | 식품접객업소 | 5천만원 이내 (화장실만 개선하는 경우는 1천만원 이내) |

        📞 문의처
        - 울산광역시청 식의약안전과 (052-229-3654)
        - 중구 환경위생과 (052-290-3732)
        - 남구 위생과 (052-226-5722)
        - 동구 환경위생과 (052-209-3572)
        - 북구 환경위생과 (052-241-7782)
        - 울주군 위생과 (052-204-2212)
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '인천광역시 동구 공고 제2024– 887호「인구감소(관심)지역 기업지원 특례보증 금융지원」신청 공고 「인구감소(관심)지역 기업지원 특례보증 금융지원」신청을 다음과 같이 공고합니다. 2024. 8. 27.인천광역시 동구청장■ 인구감소(관심)지역 기업지원 특례보증 금융지원 개요 1. 지원규모 : 총 30억원 2. 특례보증 신청대상 : 인천광역시 동구 소재 기업 중 아래 해당 기업 ○ 지역 주력산업 영위기업 및 지방이전기업 ○ 유망서비스업 영위기업 ○ 농식품분야 우수기술 보유기업 ○ 신성장 동력산업 영위기업 ○ 유망 창업기업 ○ 수출기업 및 해외 진출기업 ○ 고용창출기업 ○ 벤처기업 ○ 소재부품장비 분야 영위기업 ○ 기업가형 소상공인 ※ 동일기업 중복지원 제한없음 (중앙부처 및 지자체 지원자금 기 수혜기업) 3. 지원사항 ○ 대출한도 : 중소기업 최대 30억원, 기업가형 소상공인 최대 5억원 ※ 직전년도 매출액의 20% 이내 ○ 대출기간 : 최대 10년 (우대기간 3년) - 운전자금 : 1년 (만기 일시상환, 최대 3년) - 시설자금 : 최대 10년 (거치 및 분할상환 기간은 농협은행과 협의) ○ 지원내용 - 신용보증기금 : (특별출연 상품) 보증비율 100% 보증서(보증료 0.3% 차감, 3년간) 또는 (보증료지원 상품) 보증료 0.3% 3년간 지원 - NH농협은행 : 기업별 신용등급과 무관하게 동일 대출금리 우대지원(’24. 8. 1. 기준 1년 고정금리 적용시 대출금리 4.96%) - 인천광역시 동구 : 최종 산출금리에서 2.0% 이차보전 균등 지원(3년간)(이차보전 지원후 최종 대출금리 2.96%, `24. 8. 1. 기준) 4. 기타 조건 ○ 취급은행 : NH농협은행 인천지점 (타은행 및 농·축협 취급불가) ○ 신용보증기금 : 전국 지점 (109개) ○ 대출금리 : 협약금리 5. 추천 취소 대상 ○ 추천 결정 후 폐업, 관외 이전, 최종 부도 처리된 기업 등 ○ 허위 또는 기타 부정한 방법으로 신청 또는 대출을 받은 경우 ○ 기타 추천 요건 미충족으로 사업 추진이 불가능한 경우 6. 신청 및 접수 ○ 신청기간 : 2024. 9. 3.(화) ~ 재원 소진시 까지 ○ 신청방법 : (붙임)신청양식과 구비서류를 방문 제출 ○ 접 수 처 : 인천광역시 동구 화도진로 144(여성회관) 1층, 여성가족과 인구가족팀 ○ 신청문의 : 인천광역시 동구청 여성가족과 ☎(032)770-6491 7. 구비서류 구 분 서류 및 요건기본 서류 기업지원 특례보증서 신청서, 사업계획서, 정보제공 활용 및 이용동의서사업자등록증명원 국세청법인등기사항전부증명서 고객이 법인인 경우에 한하며, 말소된 등기사항이 기재된 법인등기사항전부증명서부가가치세 등 신고자료 최근 3개년 부가가치세과세표준증명(면세사업자의 경우 부가가치세 면세사업자 수입금액증명)재 무 제 표 최근 결산년도 재무제표(홈택스 표준재무제표증명 또는 세무사발행 재무제표확인원) ※ 상기서류는 지자체의 추천서 발급을 위한 서류로써, 신보 및 농협은행 심사서류는 별도 제출 필요8. 신청 결과 안내 ○ SMS를 통한 신청 결과 안내 및 추천서 발급(추천서 수령방법 : 방문수령)9. 특례보증 및 대출신청 절차①추천서 발급지자체 → 기업▪ 방문 수령②보증신청기업 → 신 용 보 증 기 금▪ 기업체는 신보 또는 농협은행 방문전, 보증심사 필요서류를 신보 또는 농협은행에 사전에 문의한 뒤, 추천서와 함께 필요서류 제출③대출신청기업 → 농협은행▪ 신보의 보증서 발급 후, 농협은행에 대출심사 신청 ※ 신용보증기금의 보증심사 및 농협은행의 대출심사 결과에 따라 지원 대상에서 제외될 수 있음붙임 신청서 등 관련 서식 1부. 끝.'},
        {'role' : 'assistant', 'content' : """
        📌 [인천] 동구 인구감소(관심)지역 기업지원 특례보증 금융지원 신청 공고
         
        🤖 사업 개요
        - 「인구감소(관심)지역 기업지원 특례보증 금융지원」 신청을 다음과 같이 공고합니다. 

        ✅ 사업자 유형
        - 개인사업자
        - 법인사업자
        - 기존사업자
         
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 소상공인, 대출
         
        👥 지원대상
        - 지역 주력산업 영위기업 
        - 지방이전기업
        - 유망서비스업 영위기업
        - 농식품분야 우수기술 보유기업
        - 신성장 동력산업 영위기업
        - 유망 창업기업
        - 구출기업 및 해외 진출기업
        - 고용창출기업
        - 벤처기업
        - 소재부춤장비 분야 영위기업
        - 기업가형 소상공인

        📅 신청기간
        - 2024.9.3(화) ~ 재원 소진시 까지

        📝 신청방법 
        - 신청방법 : (붙임)신청양식과 구비서류를 방문 제출
        - 접수 : 인천광역시 동구 화도진로 144(여성회관) 1층, 여성가족과 인구가족팀
         
        🚫 지원제외 대상
        - 추천 결정 후 폐업, 관외 이전, 최종 부도 처리된 기업 등
        - 허위 또는 기타 부정한 방법으로 신청 또는 대출을 받은 경우
        - 기타 추천 요건 미충족으로 사업 추진이 불가능한 경우
         
        ⭐️ 우대사항
        - 
         
        👩 여성 대상
        X
         
        💁 청년 대상
        X 
         
        ⚠ 유의사항
        - 취급은행 : NH농협은행 인천지점(타은행 및 농·축협 취급불가)
        - 신용보증기금 : 전국 지점(109개)
        - 대출금리 : 협약금리
        
        📑 제출서류
        - 기업지원 특례보증서 신청서, 사업계획서, 정보제공 활용 및 이용동의서  
        - 사업자등록증명원 (국세청)  
        - 법인등기사항전부증명서 (고객이 법인인 경우에 한하며, 말소된 등기사항이 기재된 법인등기사항전부증명서)  
        - 부가가치세 등 신고자료 (최근 3개년 부가가치세과세표준증명 또는 부가가치세 면세사업자 수입금액증명)  
        - 재무제표 (최근 결산년도 재무제표, 홈택스 표준재무제표증명 또는 세무사발행 재무제표확인원)  
                 
        📌 선정절차
        - 1. 추천서 발급: 지자체 → 기업 (방문 수령)  
        - 2. 보증신청: 기업 → 신용보증기금 (필요서류 제출)  
        - 3. 대출신청: 기업 → 농협은행 (신보의 보증서 발급 후 대출심사 신청)  
         
        💰 지원내용
        - 대출한도: 중소기업 최대 30억원, 기업가형 소상공인 최대 5억원 (직전년도 매출액의 20% 이내)  
        - 대출기간: 최대 10년 (우대기간 3년)  
        - 운전자금: 1년 (만기 일시상환, 최대 3년)  
        - 시설자금: 최대 10년 (거치 및 분할상환 기간은 농협은행과 협의)  
        - 신용보증기금: (특별출연 상품) 보증비율 100% 보증서(보증료 0.3% 차감, 3년간) 또는 (보증료지원 상품) 보증료 0.3% 3년간 지원  
        - NH농협은행: 기업별 신용등급과 무관하게 동일 대출금리 우대지원(’24. 8. 1. 기준 1년 고정금리 적용시 대출금리 4.96%)  
        - 인천광역시 동구: 최종 산출금리에서 2.0% 이차보전 균등 지원(3년간)(이차보전 지원후 최종 대출금리 2.96%, `24. 8. 1. 기준)  

        📞 문의처
        - 인천광역시 동구청 여성가족과 ☎(032)770-6491
        """}
    ]},
    {'messages': [
        {'role' : 'system', 'content' : template},
        {'role' : 'user', 'content' : '-1-충청북도 공고 제2025-90호 2025년 사 회 적 기 업 육 성 자 금 이 차 보 전 금 지 원 계 획 공 고 「사회적기업육성법」제11조,「충청북도 사회적기업 육성지원에 관한 조례」제6조에 따라 사회적기업 또는 예비사회적기업 육성자금 이차보전금 지원계획을 다음과 같이 공고합니다. 2025년 1월 22일충청북도지사Ⅰ 지원 개요❍ 지원대상 :도내 소재 사회적기업 또는 예비사회적기업❍ 지원조건 :업체당 1억원 이내,최대 5년 지원(인증3,예비2)❍ 지원횟수 :업체당 최대 2회까지 지원 가능(인증,예비 포함 총 2회)❍ 지원내용 :기업 운영 및 시설자금 대출에 대한 이자차액(2.5%)지원❍ 사 업 비 :37,822천원 -신규대출 이차보전(25,000천원)및 기존대출 이차보전(12,822천원)포함❍ 협약기관 -충청북도청 :대출금리 이자차액 보전(2.5%) -NH농협은행:협약자금(10억원)지원,최대 0.6%자체금리 할인 -신용보증기금 :신용보증서 발급 수수료 우대 적용(0.5% ),100%전액 보증Ⅱ 주요 내용【지원조건】❍융자한도 :1개 업체당,1억원 이내 ※ 자금 한도액(10억 이내)❍융자기간 :최대 5년 -예비사회적기업(2년 이내),사회적기업(3년 이내) ※ 융자받은 예비사회적기업이 사회적기업으로 전환된 경우 3년 연장-2- ❍융자금리 :은행 대출금리 –은행 최대 0.6%자체 금리할인 –도 이자 지원 2.5% < 예 시 >‣ 협약은행 대출금리가 연 4.8%라고 가정할 경우·협약은행금리최대0.6%감면·충북도이차보전금지원2.5% ⇒ 실질적으로 기업이 부담하게 되는 금리 : 1.7% 정도【지원절차】 ❍ 신청접수 및 보증 상담 :신용보증기금 ❍보증심사 및 신용보증서 발급 :신용보증기금 -신청인의 신용 및 재정상태,경영능력,사업성 등을 종합적 평가 ❍보증서 확인 및 대출 실행 :NH농협은행 ❍ 세부절차공고(도)⇨신청서 접수·보증심사(신용보증기금)⇨신용보증서 발급(신용보증기금)⇨대출실행(NH농협은행)⇨이차보전금 지급(도 → NH농협은행)【지원 제외대상】❍신청일 현재 휴·폐업 중인 업체❍ 신청일 현재 N H농협은행의 한도액까지 대출받아 상환이 완료되지 않은 기업❍ 대표자(실제경영자 등)관련 금융권 채무가 연체 중인 기업❍ 법령 위반 등으로 사회적 물의를 일으킨 기업❍ 신용보증기금 내규의 보증금지 및 보증제한 기업❍ 사회적경제기업 지역자산화 이차보전금을 지원받고 있는 기업 등Ⅲ 신청 기간 등❍ 신청기간 :2025.1.~협약자금 소진시까지 ※ 협약자금 및 이차보전금 소진 상황은 도 담당부서에서 확인 가능하며, 자금 수요에 따라 대출 중단일자는 변동될 수 있음 ❍ 신청방법 :방문 또는 온라인(해당기관 문의 후 진행) -보증심사 :신용보증기금 영업점 -대출실행:도내 NH농협은행 영업점-3-❍ 필요서류 :보증상담 및 대출신청 시 각 기관에 제출구비 목록영업점 내방 및 전화 상담 가능<공통 필요서류>➀ 예비사회적기업 지정서 또는 사회적기업 인증서② 법인등기부 등본(말소사항 포함)③ 주주명부(최신 현황)④ 임대계약서 사본(사업장 임차인 경우)⑤ 임대차계약서 사본(대표자 거주주택 임차인 경우)⑥ 재무제표(최근 3년)⑦ 부가세과세표준증명원(최근 3년)⑧ 결산부속명세서(회계사무실, ’20년~현재까지)⑨ 국민연금가입 경력증명서(대표자)⑩ 법인인감증명서(최근 3개월 이내 발급분)<기관별 추가 필요서류> ○ 보증상담(신용보증기금) : 기업실태표(일반형 또는 협동조합형)* * (양식) [신용보증기금 홈페이지] - [소식안내] - [업무서식]에서 내려받기 ○ 대출신청(NH농협) : 사업계획서(서식은 농협에 문의) ※ 위 서류 목록은 기업에 따라 달라질 수 있으며, 자세한 내용은 상담기관에 문의 바랍니다.Ⅳ 기타사항 ❍ 융자 지원 결정 후 파산·부도·폐업·휴업·회생 및 타 시·도 이전업체는 사유 발생일 전일까지만 이차보전금을 지급 ❍ 사회적기업 인증 취소나 예비사회적기업 지정기간 종료시에도 위와 같이 적용-4-Ⅴ 문의 및 접수처 ❍ 충청북도 사회적기업 담당부서 문 의 처 전화번호충청북도 소상공인정책과 사회적경제팀043)220-2573 ❍ 보증상담 및 접수(신용보증기금) 문 의 처 전화번호신용보증기금 각 영업점1588-6565(대표) ※ 인터넷 홈페이지(https://www.kodit.co.kr) 온라인 및 전화 상담 가능 ❍ 대출상담(NH농협은행) 문 의 처 전화번호NH농협은행 충북본부(대표) 043)229-1534 끝.'},
        {'role' : 'assistant', 'content' : """
        📌 [충북] 2025년 사회적기업 육성자금 이차보전금 지원계획 공고 요약 안내
         
        🤖 사업 개요
        - 「사회적기업육성법」제11조, 「충청북도 사회적기업 육성지원에 관한 조례」제6조에 따라 사회적기업 또는 예비사회적기업 육성자금 이차보전금 지원계획을 다음과 같이 공고합니다.

        ✅ 사업자 유형
        - 개인사업자
        - 법인사업자
        - 기존사업자
         
        ☑️ 지원업종
        - 자동차 및 부품 판매업, 도매 및 상품 중개업, 소매업(자동차 제외), 숙박업, 음식점업, 제조업, 교육 서비스업, 협회 및 단체, 수리 및 기타 개인 서비스업, 부동산업, 전문, 과학 및 기술 서비스업, 예술, 스포츠 및 여가관련 서비스업, 정보통신업, 농업, 임업 및 어업, 건설업, 운수 및 창고업, 보건업 및 사회복지 서비스업, 사업시설 관리, 사업 지원 및 임대 서비스업, 금융 및 보험업, 전기, 가스, 증기 및 공기 조절 공급업, 광업, 수도, 하수 및 폐기물 처리, 원료 재생업, 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동, 공공 행정, 국방 및 사회보장 행정, 국제 및 외국기관

        💡 공고특성
        - 대출
         
        👥 지원대상
        - 도내 소재 사회적기업
        - 예비사회적기업

        📅 신청기간
        - 2025.1. ~ 협약자금 소진시까지

        📝 신청방법
        - 방문 또는 온라인(해당기관 문의 후 진행)  
        - 보증심사: 신용보증기금 영업점  
        - 대출실행: 도내 NH농협은행 영업점
         
        🚫 지원제외 대상
        - 신청일 현재 휴·폐업 중인 업체
        - 신청일 현재 NH농협은행의 한도액까지 대출받아 상환이 완료되지 않은 기업
        - 대표자(실제경영자 등) 관련 금융권 채무가 연체 중인 기업
        - 법령 위반 등으로 사회적 물의를 일으킨 기업
        - 신용보증기금 내규의 보증금지 및 보증제한 기업
        - 사회적경제기업 지역자산화 이차보전금을 지원받고 있는 기업 등
         
        ⭐️ 우대사항
        - 사회적기업
         
        👩 여성 대상
        X 
         
        💁 청년 대상
        X 
         
        ⚠ 유의사항
        - 융자 지원 결정 후 파산·부도·폐업·휴업·회생 및 타 시·도 이전업체는 사유 발생일 전일까지만 이차보전금을 지급  
        - 사회적기업 인증 취소나 예비사회적기업 지정기간 종료시에도 위와 같이 적용
         
        📑 제출서류
        <공통 필요서류>
        - 예비사회적기업 지정서 또는 사회적기업 인증서
        - 법인등기부 등본(말소사항 포함)
        - 주주명부(최신 현황)
        - 임대계약서 사본(사업장 임차인 경우)
        - 임대차계약서 사본(대표자 거주주택 임차인 경우)  
        - 재무제표(최근 3년)  
        - 부가세과세표준증명원(최근 3년)  
        - 결산부속명세서(회계사무실, ’20년~현재까지)  
        - 국민연금가입 경력증명서(대표자)  
        - 법인인감증명서(최근 3개월 이내 발급분)  

        <기관별 추가 필요서류>
        - 보증상담(신용보증기금): 기업실태표(일반형 또는 협동조합형)  
        * (양식) [신용보증기금 홈페이지] - [소식안내] - [업무서식]에서 내려받기  
        - 대출신청(NH농협): 사업계획서(서식은 농협에 문의)  
        ※ 위 서류 목록은 기업에 따라 달라질 수 있으며, 자세한 내용은 상담기관에 문의 바랍니다.
    
        📌 선정절차
        - 신청접수 및 보증 상담: 신용보증기금  
        - 보증심사 및 신용보증서 발급: 신용보증기금  
            - 신청인의 신용 및 재정상태, 경영능력, 사업성 등을 종합적 평가  
        - 보증서 확인 및 대출 실행: NH농협은행  
        - 세부절차: 공고(도) ⇨ 신청서 접수·보증심사(신용보증기금) ⇨ 신용보증서 발급(신용보증기금) ⇨ 대출실행(NH농협은행) ⇨ 이차보전금 지급(도 → NH농협은행)
         
        💰 지원내용
        - 기업 운영 및 시설자금 대출에 대한 이자차액(2.5%) 지원  
        - 사업비: 37,822천원 (신규대출 이차보전 25,000천원 및 기존대출 이차보전 12,822천원 포함)  
        - 협약기관  
            - 충청북도청: 대출금리 이자차액 보전(2.5%)  
            - NH농협은행: 협약자금(10억원) 지원, 최대 0.6% 자체금리 할인  
            - 신용보증기금: 신용보증서 발급 수수료 우대 적용(0.5%), 100% 전액 보증
        
        📞 문의처
        - 충청북도 사회적기업 담당부서  
            - 충청북도 소상공인정책과 사회적경제팀: 043)220-2573  
        - 보증상담 및 접수(신용보증기금)  
            - 신용보증기금 각 영업점: 1588-6565(대표)  
            - 인터넷 홈페이지(https://www.kodit.co.kr) 온라인 및 전화 상담 가능  
        - 대출상담(NH농협은행)  
            - NH농협은행 충북본부(대표): 043)229-1534  
        """}
    ]}
]

1. 데이터셋 형식 준수 확인

In [42]:
# 딕셔너리를 만들면서 존재하지 않는 key에 대한 디폴트값 0으로 설정
format_errors = defaultdict(int)

# 데이터셋의 각 대화들이 dict 형태인지 확인
for ex in dataset : 
    if not isinstance(ex, dict) : 
        # dict 형태가 아니라면
        format_errors['data_type'] += 1
        continue

    # messages의 value값이 있는지 판단
    messages = ex.get('messages', None)
    if not messages :
        format_errors['missing_messages_list'] += 1
        continue

    # 각 대화들의 키값 확인
    for message in messages :
        if any(key not in ('role', 'content', 'name', 'function_call', 'weight') for key in message) :
            format_errors['message_unrecognized_key'] += 1

        # message에 role과 content가 있는지 다시 확인
        if 'role' not in message or 'content' not in message :
            format_errors['message_missing_key'] += 1

        if message.get('role', None) not in ('system', 'user', 'assistant') :
            format_errors['unrecognized_role'] += 1

        content = message.get('content', None)
        if not content or not isinstance(content, str) :
            format_errors['missing_content'] += 1

    # 각 대화에 assistant의 응답이 있는지 확인
    if not any(message.get('role', None) == 'assistant' for message in messages) :
        format_errors['example_missing_assistant_message'] += 1

# 발생한 에러가 있다면 에러 내용을 출력
if format_errors :
    print('에러 발견 :')
    for key, value in format_errors.items() :
        print(f"{key}: {value}")
else :
    print('에러가 없습니다 ^^')

에러가 없습니다 ^^


2. 누락 메시지 식별 및 메시지와 토큰 수 확인


In [43]:
# OpenAI 최신 LLM에 사용되는 모델 중 하나인 'cl100k_base' 사용해 인코딩
encoding = tiktoken.get_encoding('cl100k_base')

# 메시지 목록의 총 토큰 수 계산
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1) :
    num_tokens = 0
    for message in messages :
        num_tokens += tokens_per_message
        for key, value in message.items() :
            # 인코딩된 값을 토큰으로 씀
            num_tokens += len(encoding.encode(value))
            # key에 name이 있다면(assistant의 이름을 설정하면 name이 자동 생성됨)
            if key == 'name' :
                num_tokens += tokens_per_name

    # num_tokens를 3씩 증가시키는 이유 : 
        # 1) 하나의 messages 전체를 감싸는 토큰 : "{}"
        # 2) 한 message의 끝을 표시하는 특수 토큰(모델 내부에서 동작, <SEP> 등)
        # 3) 각각의 message를 구분해주는 구분 토큰 : ','
    num_tokens += 3
    return num_tokens

# assistant(모델)가 응답한 메시지의 총 토큰 수(모델의 max_output_tokens 기준에 맞는지 판단)
def num_assistant_tokens_from_messages(messages) :
    num_tokens = 0 
    for message in messages :
        if message['role'] == 'assistant' :
            # content의 value값만 토큰화+인코딩
            num_tokens += len(encoding.encode(message['content']))
    return num_tokens

# message의 토큰 길이에 대한 통계 정보 출력
def print_statistics(values) :
    print(f'min / max : {min(values)},{max(values)}')
    print(f'mean / median : {np.mean(values)}, {np.median(values)}')

In [44]:
num_assistant_tokens_from_messages(messages)

2294

In [48]:
num_tokens_from_messages(messages)

6986

In [49]:
max_output_tokens = 16384   # 모델 최대 출력 토큰 수(gpt-4o-mini의 경우 16384개)
n_missing_system = 0        # role에 system이 없는 경우의 대화 수 카운트
n_missing_user = 0          # role에 user가 없는 경우의 대화 수 카운트
n_messages = []             # 각 대화의 message 개수가 담길 빈 리스트
total_tokens_lens = []      # 각 대화의 총 토큰 수가 담길 빈 리스트
assistant_message_lens = [] # assistant가 보낸 메시지의 길이가 담길 빈 리스트

for ex in dataset :
    messages = ex['messages']
    # 각 message에서 role이 system인 경우가 하나라도 있다면 T이지만 앞에 not이 붙어서 F가 됨
    if not any(message['role']=='system' for message in messages) :
        n_missing_system += 1
    if not any(message['role']=='user' for message in messages) :
        n_missing_user += 1

    # 각 messages의 message 개수를 리스트에 추가
    n_messages.append(len(messages))
    # 메시지 목록의 총 토큰 수 함수 실행 후 리스트에 추가
    total_tokens_lens.append(num_tokens_from_messages(messages))
    # 모델 출력 토큰 수 함수 실행 수 리스트에 추가
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print('System 메시지 누락 수 :', n_missing_system)
print('User 메시지 누락 수 :', n_missing_user)

print("### 대화 당 메시지 수 통계 : ")
print_statistics(n_messages)

print("### 대화 당 전체 토큰 수 통계 : ")
print_statistics(total_tokens_lens)

print("### 대화 당 assistant 수 통계 : ")
print_statistics(assistant_message_lens)

# 각 메시지들의 총 토큰 수 리스트에서 각각의 값을 16384와 비교하여 넘어가는지 판단
n_too_long = sum(i > max_output_tokens for i in total_tokens_lens)
print(f"\n{n_too_long}개의 대화가 {max_output_tokens}개 토큰 제한을 초과하며 이 부분은 학습 중 잘릴 수 있습니다.")

System 메시지 누락 수 : 0
User 메시지 누락 수 : 0
### 대화 당 메시지 수 통계 : 
min / max : 3,3
mean / median : 3.0, 3.0
### 대화 당 전체 토큰 수 통계 : 
min / max : 2999,15498
mean / median : 6918.3, 6068.0
### 대화 당 assistant 수 통계 : 
min / max : 913,3326
mean / median : 1716.4, 1568.5

0개의 대화가 16384개 토큰 제한을 초과하며 이 부분은 학습 중 잘릴 수 있습니다.


In [50]:
# 모델에 맞는 대화당 최대 토큰 수 설정
MAX_TOKENS_PER_EXAMPLE = 15000

# 학습 횟수 설정
TARGET_EPOCHS = 10
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 15
MIN_TARGET_EXAMPLES = 10
MAX_TARGET_EXAMPLES = 10

n_epochs = 3
n_train_examples = len(dataset)

# 대화의 개수와 TARGET_EPOCHS의 곱이 최소 데이터 수 보다 작다면
if(n_train_examples * TARGET_EPOCHS) < MIN_TARGET_EXAMPLES:
    # OpenAI 테스트 기준 epochs 최대치(25회)와 최소 데이터(100개)에서 우리 데이터 수를 나눈 몫을 비교하여
    # 더 낮은 값으로 최종 학습 횟수(n_epochs) 설정
    n_epochs = min(MAX_DEFAULT_EPOCHS, (MIN_TARGET_EXAMPLES // n_train_examples))

elif (n_train_examples * TARGET_EPOCHS) > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, (MAX_TARGET_EXAMPLES // n_train_examples))
    
# 데이터 셋에 있는 모든 메시지의 총 토큰 수를 합산
n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in total_tokens_lens)

print(f'데이터 셋에는 학습 중 요금이 청구될 {n_billing_tokens_in_dataset}개의 토큰이 있습니다.')
print(f'기본적으로 이 데이터 셋에서 {n_epochs} epoch 동안 학습합니다.')
print(f'총 {n_epochs*n_billing_tokens_in_dataset}개의 토큰에 대해 요금이 청구 됩니다.')

데이터 셋에는 학습 중 요금이 청구될 68685개의 토큰이 있습니다.
기본적으로 이 데이터 셋에서 1 epoch 동안 학습합니다.
총 68685개의 토큰에 대해 요금이 청구 됩니다.


In [ ]:
# # fine-tuning을 위한 파일 생성 객체 만들기
# fine_tune_files = client.files.create(
#     file = open("finetune_data/bizinfo.dataset.jsonl", "rb"),
#     purpose = "fine-tune"
# )

# fine_tune_files

In [ ]:
# # 작업 객체 생성
# fine_tune_job = client.fine_tuning.jobs.create(
#     model = 'gpt-4o-mini-2024-07-18',
#     # fine_tune_files(데이터 파일 객체)의 id값 지정
#     training_file = fine_tune_files.id,
# )

# fine_tune_job

In [ ]:
# 작업 세부 사항 확인
client.fine_tuning.jobs.retrieve("ftjob-QFOgzTXXJfhK1BK2clcWiT8i")

In [56]:
# fine-tuning 완료 후 질의응답
completion = client.chat.completions.create(
    model = "ft:gpt-4o-mini-2024-07-18:fininsight::BFAp1Zth",
    messages = [{"role":"system", "content": template},
                {'role' : 'user', 'content' : cleaned_text}]
    )

print(completion.choices[0].message.content)


📌 [경남] 2025년 1인 자영업자 사회보험망(고용·산재보험료) 지원사업 공고 요약 안내

🤖 사업 개요
- 경상남도 내 1인 자영업자에 대한 고용보험료 및 산재보험료 지원을 통해 폐업과 산업재해 등으로부터 보호하고 사회안전망을 강화하기 위하여 2025년도 「소상공인 사회보험망 지원사업」을 공고하오니 많은 참여 바랍니다.

✅ 사업자 유형
- 개인사업자
- 법인사업자
- 기존사업자

☑️ 지원업종
- 자동차 및 부품 판매업
- 도매 및 상품 중개업
- 소매업(자동차 제외)
- 숙박업
- 음식점업
- 제조업
- 교육 서비스업
- 협회 및 단체, 수리 및 기타 개인 서비스업
- 부동산업
- 전문, 과학 및 기술 서비스업
- 예술, 스포츠 및 여가관련 서비스업
- 정보통신업
- 농업, 임업 및 어업
- 건설업
- 운수 및 창고업
- 보건업 및 사회복지 서비스업
- 사업시설 관리, 사업 지원 및 임대 서비스업
- 금융 및 보험업
- 전기, 가스, 증기 및 공기 조절 공급업
- 광업
- 수도, 하수 및 폐기물 처리, 원료 재생업
- 가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동
- 공공행정, 국방 및 사회보장 행정
- 국제 및 외국기관

💡 공고특성
- 소상공인
- 고용지원
- 대출

👥 지원대상
- 고용보험 및 산재보험에 가입하고, 보험료를 납부한 실적이 있는 도내 1인 자영업자
※ 사업자등록증상 공동사업자의 경우 대표 1인에 대해서만 지원

📅 신청기간
- 2025.2.12 ~ 12.31 연중 수시 신청

📝 신청방법
1. 경남바로서비스(https://baro.gyeongnam.go.kr/baro/) 접속
2. ‘1인 자영업자 고용·산재보험 가입지원 사업’ 배너 클릭
3. ‘신청’ 클릭
4. ‘신청하기’ 클릭
5. 로그인(회원가입 및 휴대폰인증 필수)
6. 개인정보 수집·이용 동의서 체크 후 ‘동의’ 클릭
7. 신청구분, 체크리스트, 주민등록번호, 사업자등록번호, 지급계좌 등 입력 후 ‘마이데이터가져오기’ 클릭
8. 추가서류 제출이 필요한 경우 ‘